In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
==> aki so pra gerar erro de clicar em executar tudo

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/pesquisa')

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

from datetime import datetime

from str_utils import arquivos_amostras, busca_arquivos, quebra_nome_arquivo, gravar_arquivo_saida, df_amostras, tipo_nucleotideo_map, labels_amostras, find_str, shannon_entropy, gravar_arquivo_saida, analises, pasta_analise, hamming_distance

1 gerar diagrama de sequencia do processo

2 Por que STRs são tão eficazes?

    Alta variabilidade entre indivíduos

    Baixa taxa de mutação

    Facilidade de amplificação por PCR

    Compatibilidade com sistemas multiplex, que analisam vários loci simultaneamente

In [10]:
%%writefile /content/drive/MyDrive/pesquisa/str_utils.py

import os
import pandas as pd
import math
import datetime

analises = [1, 2, 3, 4, 5, 6, 7, 8, 9]
diretorio_saida = "/content/drive/MyDrive/pesquisa/"
pasta_analise = ""
pasta_analise = "/content/drive/MyDrive/pesquisa/2025-09-04/"

labels_amostras = ['ASM285v2', 'ASM1928827v1', 'ASM2978390v1', 'ASM2978392v1', 'ASM4765177v1']
arquivos_amostras = [
  '/content/drive/MyDrive/fasta/aspergillus/ASM285v2/ncbi_dataset/data/GCA_000002855.2/GCA_000002855.2_ASM285v2_genomic.fna',
  '/content/drive/MyDrive/fasta/aspergillus/ASM1928827v1/ncbi_dataset/data/GCA_019288275.1/GCA_019288275.1_ASM1928827v1_genomic.fna',
  '/content/drive/MyDrive/fasta/aspergillus/ASM2978390v1/ncbi_dataset/data/GCA_029783905.1/GCA_029783905.1_ASM2978390v1_genomic.fna',
  '/content/drive/MyDrive/fasta/aspergillus/ASM2978392v1/ncbi_dataset/data/GCA_029783925.1/GCA_029783925.1_ASM2978392v1_genomic.fna',
  '/content/drive/MyDrive/fasta/aspergillus/ASM4765177v1/ncbi_dataset/data/GCA_047651775.1/GCA_047651775.1_ASM4765177v1_genomic.fna'
]

# dicionário Montagem -> Referência GenBank
tipo_nucleotideo_map = {
    "mononucleotídeo": 1,
    "dinucleotídeo": 2,
    "trinucleotídeo": 3,
    "tetranucleotídeo": 4,
    "pentanucleotídeo": 5,
    "hexanucleotídeo": 6,
    "heptanucleotídeo": 7,
    "octanucleotídeo": 8,
    "nonanucleotídeo": 9
}

ASM285v2_scaffolds_validos = [
    "NT_166518", "NT_166519", "NT_166520", "NT_166521",
    "NT_166522", "NT_166523", "NT_166524", "NT_166525",
    "NT_166526", "NT_166527", "NT_166528", "NT_166529",
    "NT_166530", "NT_166531", "NT_166532", "NT_166533",
    "NT_166537", "NT_166538", "NT_166539"
]

# Criar dados como lista de listas
dados = [
    ["ASM285v2",     "ASM285v2 *",   "GCA_000002855.2", "PRJNA19275", "CBS 513.88",   ASM285v2_scaffolds_validos,   "DSM, The Netherlands",    "Scaffold",   34, '/content/drive/MyDrive/fasta/aspergillus/ASM285v2/ncbi_dataset/data/GCA_000002855.2/GCA_000002855.2_ASM285v2_genomic.fna'],
    ["ASM1928827v1", "ASM1928827v1", "GCA_019288275.1", "JAGRPH01",   "CBS 554.65",   [], "TU Wien",                 "Chromosome", 40, '/content/drive/MyDrive/fasta/aspergillus/ASM1928827v1/ncbi_dataset/data/GCA_019288275.1/GCA_019288275.1_ASM1928827v1_genomic.fna'],
    ["ASM2978390v1", "ASM2978390v1", "GCA_029783905.1", "JAPVRD01",   "KJC3",         [], "Soongsil University",     "Chromosome", 40, '/content/drive/MyDrive/fasta/aspergillus/ASM2978390v1/ncbi_dataset/data/GCA_029783905.1/GCA_029783905.1_ASM2978390v1_genomic.fna'],
    ["ASM2978392v1", "ASM2978392v1", "GCA_029783925.1", "JAPVRE01",   "KYF3",         [], "Soongsil University",     "Chromosome", 37, '/content/drive/MyDrive/fasta/aspergillus/ASM2978392v1/ncbi_dataset/data/GCA_029783925.1/GCA_029783925.1_ASM2978392v1_genomic.fna'],
    ["ASM4765177v1", "ASM4765177v1", "GCA_047651775.1", "JBKZXA01",   "CCTCC 206047", [], "Zhejiang Uni.Technology", "Complete",   35, '/content/drive/MyDrive/fasta/aspergillus/ASM4765177v1/ncbi_dataset/data/GCA_047651775.1/GCA_047651775.1_ASM4765177v1_genomic.fna']
]

# Criar DataFrame
colunas = [
    "id",
    "Nome da Montagem",
    "Referência GenBank",
    "WGS Accession",
    "Cepa (strain)",
    "Scaffolds",
    "Submissor",
    "Nível de montagem",
    "Tamanho (Mb)",
    "Dados"
]

df_amostras = pd.DataFrame(dados, columns=colunas)
# print(df_amostras)



if len(pasta_analise) <= 0:
  data_hora = datetime.datetime.now()
  data_hora = data_hora.strftime('%Y-%m-%d')
  pasta_analise = diretorio_saida + data_hora + '/'
  # Criar a pasta se não existir
  if not os.path.exists(pasta_analise):
      os.makedirs(pasta_analise)
      print(f"Pasta criada: {pasta_analise}")
  else:
      print(f"A pasta '{pasta_analise}' já existe!")



def gravar_arquivo_saida(nome_amostra, tamanho_str, tipo, dados):
  url_arquivo = pasta_analise + nome_amostra + '_' + tipo+ '_' + str(tamanho_str) + '.csv'
  if os.path.exists(url_arquivo):
    os.remove(url_arquivo)
  dados.to_csv(url_arquivo, sep=';', index=False)
  # print(url_arquivo)
  return f'{url_arquivo} ({len(dados)})'


# buscar aquivos na pasta de analises. tipos de arquivos: STR
def busca_arquivos(tipo, extensao="csv", ordenar_por="nome", reverso=False):
  arquivos_localizados = []
  # Percorrendo todos os diretórios e subdiretórios
  for raiz, _, arquivos in os.walk(pasta_analise):
      for nome_arquivo in arquivos:
          # Aplicar filtros
          if nome_arquivo.endswith(f".{extensao}") and tipo in nome_arquivo:
          # if nome_arquivo.endswith(".txt") and tipo in nome_arquivo:
              caminho_arquivo = os.path.join(raiz, nome_arquivo)  # Caminho completo do arquivo
              # nome_pasta = os.path.basename(raiz)[0:5]  # Nome da pasta onde o arquivo está
              arquivos_localizados.append(caminho_arquivo)

  if ordenar_por == "nome":
      arquivos_localizados.sort(key=lambda x: os.path.basename(x), reverse=reverso)
  elif ordenar_por == "data":
      arquivos_localizados.sort(key=lambda x: os.path.getmtime(x), reverse=reverso)
  elif ordenar_por == "tamanho":
      arquivos_localizados.sort(key=lambda x: os.path.getsize(x), reverse=reverso)

  return arquivos_localizados
# busca_arquivos('STR')


def quebra_nome_arquivo(arquivo):
    nome_arquivo = os.path.basename(arquivo) # Obter apenas o nome do arquivo sem o diretório
    parte_principal = os.path.splitext(nome_arquivo)[0] # Remover a extensão do arquivo
    return parte_principal.split('_')
# quebra_nome_arquivo(busca_arquivos('STR')[0])



def shannon_entropy(sequence):
    from collections import Counter
    # Conta as ocorrências de cada base
    freq = Counter(sequence)
    total = len(sequence)

    # Calcula a entropia
    entropy = 0
    for base in freq:
        p = freq[base] / total
        entropy -= p * math.log2(p)

    return entropy

# Exemplo
# dna_seq = "TAATAT"
# entropy = shannon_entropy(dna_seq)
# print(f"Entropia de Shannon: {entropy:.4f}")

# testando STRs em tandem
def find_str(sequencia_dna, tamanho_str):
  lista = []
  fim_da_fila = 0
  str_localizado = False

  for i in range(len(sequencia_dna)):
    if (str_localizado and fim_da_fila > 0 and i < fim_da_fila) or (sequencia_dna[i] not in ['A', 'C', 'T', 'G']):
      continue

    amostra_str = ''
    fim_da_fila = 0
    if i < len(sequencia_dna)-tamanho_str+1:
      inicio_amostra_str = i
      fim_amostra_str = inicio_amostra_str+tamanho_str
      amostra_str = sequencia_dna[inicio_amostra_str:fim_amostra_str]

      quantidade_sequencias_repetidas = 0
      is_testar_proxima_Sequencia = True
      while is_testar_proxima_Sequencia:
        inicio_proxima_sequencia = i+(tamanho_str*(quantidade_sequencias_repetidas+1))
        fim_proxima_sequencia = inicio_proxima_sequencia+tamanho_str
        proxima_sequencia = ''
        if fim_proxima_sequencia <= len(sequencia_dna):
          proxima_sequencia = sequencia_dna[inicio_proxima_sequencia:fim_proxima_sequencia]

        if amostra_str == proxima_sequencia:
          quantidade_sequencias_repetidas = quantidade_sequencias_repetidas+1
          fim_da_fila = fim_proxima_sequencia
        else:
          is_testar_proxima_Sequencia = False
      # fim while

      if quantidade_sequencias_repetidas > 0:
        str_localizado = True
        lista.append({
              'posicao': i+1,
              'unidade': amostra_str,
              'inicio_loci': inicio_amostra_str,
              'fim_unidade': fim_amostra_str,
              'fim_loci': fim_da_fila,
              'copias': quantidade_sequencias_repetidas +1
        })
      else:
        str_localizado = False
  # fim for
  df = pd.DataFrame.from_records(lista,index=['posicao'])
  return df

def busca_sequencias_fasta():
  from Bio.SeqIO.FastaIO import SimpleFastaParser

  sequencias = []
  for idx_amostras, arquivo in enumerate(arquivos_amostras):
    sequencias_arquivo_sfp = ''
    with open(arquivo) as handle:
      for seq_id, seq in SimpleFastaParser(handle):
        sequencias_arquivo_sfp += str(seq).upper()
    sequencias.append(sequencias_arquivo_sfp)
    print(f'amostra lida: {labels_amostras[idx_amostras]} - {str(len(sequencias[1]))}')
  return sequencias

def busca_sequencia_fasta(origem):
  from Bio.SeqIO.FastaIO import SimpleFastaParser

  resultado = df_amostras.query('id == @origem')
  if resultado.empty:
      raise ValueError(f'Origem não encontrada: "{origem}"')
  arquivo = resultado['Dados'].iloc[0]

  sequencias_arquivo_sfp = ''
  with open(arquivo) as handle:
    # print(f' > lido: {arquivo} - ({len(arquivo)})')
    for seq_id, seq in SimpleFastaParser(handle):
      sequencias_arquivo_sfp += str(seq).upper()
  # print(f' > retorno [0:25]: {sequencias_arquivo_sfp[0:25]}')
  return sequencias_arquivo_sfp


def busca_sequencia_fasta_com_filtro(origem, target_contig=None):
    resultado = df_amostras.query('id == @origem')
    if resultado.empty:
        raise ValueError(f'Origem não encontrada: "{origem}"')
    arquivo = resultado['Dados'].iloc[0]

    sequencias_alvo = ''
    with open(arquivo) as handle:
        for seq_id, seq in SimpleFastaParser(handle):
            if target_contig is None or target_contig in seq_id:
                sequencias_alvo += str(seq).upper()
    return sequencias_alvo



def create_flank(dna_sequence, inicio, fim, tamanho_fatias=200):
    # Garante que os índices estão dentro dos limites da sequência
    flank_length = max(tamanho_fatias, fim - inicio -1)

    # Fragmento central
    fragment = dna_sequence[inicio:fim]

    # Flanqueamento anterior (forward)
    inicio_forward = max(0, inicio - flank_length)
    forward_flank = dna_sequence[inicio_forward:inicio]
    # print(f'inicio_forward: {inicio_forward} forward_flank: {forward_flank} flank_length: {flank_length}')

    # Flanqueamento posterior (reverse)
    fim_reverse = min(fim + flank_length, len(dna_sequence))
    reverse_flank = dna_sequence[fim:fim_reverse]
    # print(f'fim_reverse: {fim_reverse} reverse_flank: {reverse_flank} flank_length: {flank_length}')

    # Bordas adicionais para contexto
    inicio_amostral = max(0, inicio_forward - tamanho_fatias)
    fim_amostral = min(fim_reverse + tamanho_fatias, len(dna_sequence))
    border_left = dna_sequence[inicio_amostral:inicio_forward]
    border_right = dna_sequence[fim_reverse:fim_amostral]


    return {
        "inicio_amostral": inicio_amostral,
        "fim_amostral": fim_amostral,
        "fragment": fragment,
        "montante": forward_flank,
        "jusante": reverse_flank,
        "full_flank": forward_flank + fragment + reverse_flank,
        "borda_esquerda": border_left,
        "borda_direita": border_right
    }




def dna_complementar(seq):
    complemento = {
        'A': 'T',
        'T': 'A',
        'C': 'G',
        'G': 'C'
    }
    complementar = ''
    for base in seq.upper():
        complementar += complemento.get(base, base)
    return complementar

def dna_complementar_inverso(seq):
    complementar_inverso = dna_complementar(''.join(reversed(seq)))
    return complementar_inverso



# descontinuada por busca_melhor_alinhamento
def localiza_primers(amostra, loc, inv=False):
  # print(f'=> {loc}') dna_complementar
  x = -1
  y = 0
  pos = amostra.find(loc)
  if inv:
    # o primer right esta é base complementar e esta invertido para manter o 5'->3'
    # print(f'primer: {loc}')
    comp = dna_complementar(loc)
    # print(f' comp: {comp}')
    reverse = ''.join(reversed(comp))
    # print(f' reverse: {reverse}')
    pos = amostra.find(dna_complementar(''.join(reversed(loc))))
    if not pd.isna(pos) and pos != -1:
        y = 1
        x = pos
        fim = pos + len(loc)
        resultado = amostra[pos:fim]
        # print(f'Amostra REVERSA localizada: {resultado} de {pos} a {fim}')
    if y == 0:
      print("Amostra REVERSA não localizada com os primers.")
  else:
    if not pd.isna(pos) and pos != -1:
        y = 1
        x = pos
        fim = pos + len(loc)
        resultado = amostra[pos:fim]
        # print(f'Amostra DIRETA localizada: {resultado} de {pos} a {fim}')
    if y == 0:
      print("Amostra DIRETA não localizada com os primers.")
  return x



def hamming_distance(string_1, string_2):
  if len(string_1) > len(string_2):
    for i in range(len(string_1) - len(string_2)):
      string_2 += ' '
  if len(string_2) > len(string_1):
    for i in range(len(string_2) - len(string_1)):
      string_1 += ' '
  diferencas = sum(x != y for x, y in zip(string_1, string_2))
  return diferencas / len(string_1)


# nova VERSAO ajustada abaixo
# procura melhor alinhamento em uma amostra
#  qual a posicao se encaixa melhor na amostra
#    distancia == 1 quer dizer nenhum alinhamento, neste caso devolve índice -1
#    => o primer_left precisa estar alinhado à esquerda do STR (meio para traz)
#    => o primer_right precisa estar alinhado à direita do STR (meio até o fim)
#   ________________________
def busca_melhor_alinhamento(amostra, localizar, reverse=False):
  menor_dh = 1000.9
  recorte = ''
  idx = -1

  inicio_consulta = 0
  fim_consulta    = len(amostra)

  if reverse:
    localizar_inv_compl = dna_complementar_inverso(localizar)
    print(f' >> localizar foi invertido e complementado: de {localizar} para {localizar_inv_compl}')
    localizar = localizar_inv_compl

  # print(f'consultar: "{localizar}" de: {inicio_consulta} a: {fim_consulta} amostra: {len(amostra)}')

  for i in range(inicio_consulta, fim_consulta):
    corte = amostra[i:i + len(localizar)]
    dh = hamming_distance(localizar.upper(), corte.upper())
    # print(f'{i}: {corte} - {dh}')
    if dh < menor_dh:
      menor_dh = dh
      idx = i
  # print(f'menor distancia: {menor_dh} idx: {idx}' )
  if menor_dh != 1.0:
    recorte = amostra[idx:idx + len(localizar)]
    # print(f' > menor que 1: {menor_dh} idx: {idx} recorte: {recorte}' )
    return {'distancia': menor_dh, 'indice': idx, 'recorte': recorte, 'localizar': localizar}
  else:
    return {'distancia': -1, 'indice': -1, 'recorte': '', 'localizar': ''}

# def busca_melhor_alinhamento(amostra, localizar, reverse=False):
#     menor_dh = float('inf')
#     idx = -1
#     recorte = ''

#     if reverse:
#         localizar = dna_complementar_inverso(localizar)

#     for i in range(len(amostra) - len(localizar) + 1):
#         corte = amostra[i:i + len(localizar)]
#         dh = hamming_distance(localizar.upper(), corte.upper())
#         if dh < menor_dh:
#             menor_dh = dh
#             idx = i

#     if menor_dh < len(localizar):  # encontrou algum alinhamento
#         recorte = amostra[idx:idx + len(localizar)]
#         similaridade = 1 - (menor_dh / len(localizar))
#         return {
#             'distancia': menor_dh,
#             'indice': idx,
#             'recorte': recorte,
#             'localizar': localizar,
#             'similaridade': similaridade
#         }
#     else:
#         return {'distancia': -1, 'indice': -1, 'recorte': '', 'localizar': ''}

Overwriting /content/drive/MyDrive/pesquisa/str_utils.py


In [ ]:
# PASSO 1 - Analise unidade de repetição com tamanhos de 1 a 9
# leitura dos arquivos fasta bigData e geração dos STRs para cada amostra (arquivo)
# !pip install biopython
data1 = datetime.now()
print(f'>>> inicio - {data1.strftime("%Y-%m-%d %H:%M:%S").replace(" ", "_").replace(":", "-")}')

from Bio.SeqIO.FastaIO import SimpleFastaParser

for idx_amostras, arquivo in enumerate(arquivos_amostras):
  # print(arquivo)
  sequencias_arquivo_sfp = ''
  with open(arquivo) as handle:
    for seq_id, seq in SimpleFastaParser(handle):
      sequencias_arquivo_sfp += str(seq).upper()

  print(f'   > lido: {arquivo}')

  for idx, tamanho_analise in enumerate(analises):
    analise_str = find_str(sequencias_arquivo_sfp, tamanho_analise)
    arq = gravar_arquivo_saida(labels_amostras[idx_amostras], tamanho_analise, 'STR', analise_str)
    print(f'   > gravado: {arq}')

data2 = datetime.now()
print(f'>>> fim - {data2.strftime("%Y-%m-%d %H:%M:%S").replace(" ", "_").replace(":", "-")} - {data2 - data1}')

In [ ]:
# PASSO 1.1 => TESTES de confirmação do PASSO 1
# !pip install biopython

from Bio.SeqIO.FastaIO import SimpleFastaParser

amostra_selecionada = df_amostras.iloc[0]
print(amostra_selecionada.id)

arquivo = str(amostra_selecionada['Dados'])
print(arquivo)

sequencia_arquivo = ''
with open(arquivo) as handle:
  for seq_id, seq in SimpleFastaParser(handle):
    sequencia_arquivo += str(seq).upper()
print(f'amostra lida: {amostra_selecionada.id}: {sequencia_arquivo[:100]}')

arquivo_str = f'{pasta_analise}ASM285v2_STR_4.csv'
df_str_levantados = pd.read_csv(arquivo_str, delimiter=";")
print(f'{len(df_str_levantados)}')

qt_amostras_divergentes = 0
for idx, str_levantado in df_str_levantados.iterrows():
  fragmento = str(str_levantado.unidade * str_levantado.copias)
  str_localizado_amostra              = sequencia_arquivo[str_levantado.inicio_loci     :str_levantado.fim_loci]
  str_localizado_amostra_com_bordas10 = sequencia_arquivo[str_levantado.inicio_loci - 10:str_levantado.fim_loci + 10]

  # print(f'{str_levantado.unidade} inicio_loci: {str_levantado.inicio_loci} fim_unidade: {str_levantado.fim_unidade} fim_loci: {str_levantado.fim_loci} copias: {str_levantado.copias}')
  # print(f'    sequencia original....: {str_localizado_amostra}')
  # print(f'    sequencia fragmento...: {fragmento}')
  # print(f'    sequencia +10 posições: {str_localizado_amostra_com_bordas10}')
  if str_localizado_amostra != fragmento:
    qt_amostras_divergentes = qt_amostras_divergentes + 1
    print(f'>>>>>> amostras diferentes original/str: {str_localizado_amostra}/{str_localizado_amostra_com_bordas10}')
print(f'>>> fim ({qt_amostras_divergentes} fragmentos de STR divergentes da amostra original nas mesmas posições)')



In [ ]:
# Passo 2 - Remover Duplicados (referência p artigo?) 10-09-2025
#   => gera dois arquivos:
#       1) _TIPO_ que sao os _STR_ sem as duplicidades de locus
#       2) _DIFF_ que sao os _STR_ duplicados em mesmo locus e removidos no _TIPO_
#   => procura por 'STRs dentro de STR' o que indica presença em um STR mais curto sendo replicado em um STR longo (AA em A, GTA em GTAGTAGTA...)
#      busca arquivos de STR levantados de uma amostra com tipos > 1 e analisa str
#   => performance: tabela de CONTENCAO e entropia de shannon

# antigo: ds_sel_filtro = df_final.loc[df_final.groupby('inicio')['tamanho_unidade'].idxmin()]
# remover inicialmente os de mesmo inicio_loci ?? para melhorar performance

import pandas as pd
import time
import shutil

def processar_sub_str(arquivo_str):
    t0 = time.perf_counter()

    CONTENCAO = {
        9: [3, 1],
        8: [4, 2, 1],
        7: [1],
        6: [3, 2, 1],
        5: [1],
        4: [2, 1],
        3: [1],
        2: [1],
        1: []
    }

    id_arquivo = quebra_nome_arquivo(arquivo_str)
    tipo = int(id_arquivo[2])
    origem = id_arquivo[0]

    # testando tipo de STR 1 para copiar direto o arquivo completo sem precisar ler/gravar no pandas
    if tipo == 1: # mantem todos os STRs do tipo 1, sem filtro
      copied_file = f'{pasta_analise}{origem}_TIPO_{tipo}.csv'
      shutil.copy(arquivo_str, copied_file)
      tempo = time.perf_counter() - t0
      tempo_formatado = time.strftime("%H:%M:%S", time.gmtime(tempo)) + f".{int((tempo % 1) * 1000):03d}"
      print(f'   > gravado: {copied_file} ###sem leitura no pandas #tempo: {tempo_formatado}')
      return

    df_str_levantados = pd.read_csv(arquivo_str, delimiter=";")
    print(f'   > lido {arquivo_str} ({len(df_str_levantados)})')

    # print(f' > arquivo: {arquivo_str} - origem: {origem} - tipo: {tipo} => {len(df_str_levantados)}')

    df_str_levantados['entropy'] = df_str_levantados['unidade'].apply(shannon_entropy)
    df_str_levantados['tipo'] = tipo
    df_str_levantados['origem'] = origem

    # if tipo == 1: # mantem todos os STRs do tipo 1, sem filtro
    #   arq = gravar_arquivo_saida(origem, tipo, 'TIPO', df_str_levantados)
    #   tempo = time.perf_counter() - t0
    #   tempo_formatado = time.strftime("%H:%M:%S", time.gmtime(tempo)) + f".{int((tempo % 1) * 1000):03d}"
    #   print(f'   > gravado: {arq} #diferença: {len(df_str_levantados) - len(df_str_levantados)} () tempo: {tempo_formatado}')
    #   return

    # df_str_levantados = df_str_levantados.reset_index(drop=True)
    ultimo_str_processado = ''
    ultimo_str_processado_loc = False
    indices_mantidos = []
    sub_str_localizado = False

    # ordenar por unidade e copias e evitar chamar find_str para mesma sequencia anteiror
    df_str_ordenado = df_str_levantados.sort_values(by=["unidade"], ascending=[True])
    # print(df_str_ordenado)

    for idx_str_levantados, str_levantado in df_str_ordenado.iterrows():
      str_levantamento = str(str_levantado['unidade'])
      # print(f'ultimo_str_processado: {str(ultimo_str_processado)} == str_levantamento: {str_levantamento} > origem: {origem}-{idx_str_levantados}')

      if ultimo_str_processado == str_levantamento:
        sub_str_localizado = ultimo_str_processado_loc
      else:
        sub_str_localizado = False
        for tipo_str_contencao in CONTENCAO.get(tipo, []):
          # if int(tipo_str_contencao) == 1:
          if str_levantado['entropy'] == 0: # performance
            # print(f'str_levantamento: {str_levantamento} #tipo: {tipo} #unidade: {str_levantado['unidade']} #entropia: {str_levantado['entropy']} #tipo_str_contencao: {tipo_str_contencao}')
            sub_str_localizado = True # se entropia é 0 (AAAAA por exemplo) este sempre estará em um analise tipo/tamanho 1
          # else:
          #   sub_str_localizado = False # se entropia é direfente de 0 (AAAAC por exemplo) quer dizer que ele nao será um STR do tipo 1
            break # nos dois casos acima dele deve dispensar esse registro, salta o tratamento abaixo e vai para próximo registro

          sub_strs = find_str(str_levantamento, tipo_str_contencao)
          if not sub_strs.empty:
            for _, sub_str in sub_strs.iterrows():
              amostra_sub_str = sub_str['unidade'] * sub_str['copias']
              if str_levantamento == amostra_sub_str:
                sub_str_localizado = True
                # print(f'str_levantamento: {str_levantamento} #tipo: {tipo} #unidade: {str_levantado['unidade']} #unidade_sub_str: {sub_str['unidade']}-{sub_str['copias']} #entropia: {str_levantado['entropy']} #tipo_str_contencao: {tipo_str_contencao}')
                break
          if sub_str_localizado:
            break
      if not sub_str_localizado:
        indices_mantidos.append(idx_str_levantados)

      ultimo_str_processado = str_levantamento
      ultimo_str_processado_loc = sub_str_localizado

    df_sub = df_str_ordenado.loc[indices_mantidos]
    arq = gravar_arquivo_saida(origem, tipo, 'TIPO', df_sub)
    tempo = time.perf_counter() - t0
    tempo_formatado = time.strftime("%H:%M:%S", time.gmtime(tempo)) + f".{int((tempo % 1) * 1000):03d}"
    print(f'   > gravado: {arq} #diferença: {len(df_str_levantados) - len(df_sub)} ({len(df_str_levantados)}-{len(df_sub)}) tempo: {tempo_formatado}')

# principal
t_principal = time.perf_counter()
for idx_label, label_amostra in enumerate(labels_amostras):
  print(f'> amostra: {label_amostra} ({idx_label + 1}) de {len(labels_amostras)}')
  nome_arq = label_amostra + '_STR'
  arquivos_str = busca_arquivos(nome_arq, ordenar_por="nome", reverso=True)

  for idx_arquivo, arquivo in enumerate(arquivos_str):
    print(f'> processar arquivo: {arquivo}')
    processar_sub_str(arquivo)

tempo_final = time.perf_counter() - t_principal
tempo_final_formatado = time.strftime("%H:%M:%S", time.gmtime(tempo_final)) + f".{int((tempo_final % 1) * 1000):03d}"
print(f'>>> fim ({tempo_final_formatado})')

In [ ]:
# PASSO 3 - soma as repeticoes - cria arquivos sumarizados (com repeticoes):
#  🔹 _REP1_ => geral - tamanho do dataFrame de cada tipo (tamanho)
#  🔹 _REP2_ => por unidade: agrupamento dos STRs (unidade) - quantidade de cada unidade str por tipo (tamanho)
#  🔹 _REP3_ => por unidade e tamanho: agrupamento dos STRs (unidade) de mesmo tamanho (copias) em cada tipo (tamanho)
#
# 11-09-20025
#
# .
# leitura dos arquivos com os STRs filtrados e geração de arquivos com group_by
# repetições
#   > soma o numero de repeticoes (unidade e copias) na mesma amostra

arquivos_str = busca_arquivos('TIPO')

for idx, arquivo in enumerate(arquivos_str):
  id_arquivo = quebra_nome_arquivo(arquivo)
  result_csv = pd.read_csv(arquivo, delimiter = ";")
  print(f'> lido: {arquivo} ({len(arquivo)})')

  dados_geral = {
    "origem": id_arquivo[0],
    "tipo": id_arquivo[2],
    "repeticoes": len(result_csv)
  }
  df_geral = pd.DataFrame([dados_geral])


  df_rep = result_csv.groupby(by=['unidade']).size().reset_index(name='repeticoes')
  df_rep['origem'] = id_arquivo[0]
  df_rep['tipo'] = id_arquivo[2]

  df_rep_copias = result_csv.groupby(by=['unidade', 'copias']).size().reset_index(name='repeticoes')
  df_rep_copias['origem'] = id_arquivo[0]
  df_rep_copias['tipo'] = id_arquivo[2]

  arq1 = gravar_arquivo_saida(id_arquivo[0], id_arquivo[2], 'REP1', df_geral)
  print(f'  > gravado: {arq1}')

  arq2 = gravar_arquivo_saida(id_arquivo[0], id_arquivo[2], 'REP2', df_rep)
  print(f'  > gravado: {arq2}')

  arq3 = gravar_arquivo_saida(id_arquivo[0], id_arquivo[2], 'REP3', df_rep_copias)
  print(f'  > gravado: {arq3}')

print('>>> fim')

In [ ]:
# PASSO 3.1 - agrupamento dos arquivos gerados no PASSO 3 - gera arquivoo SOMA_1
#   - quantidade de STRs localizados para cada tipo de STR (tamanho) de cada amostra

# REP1 => total geral: quantos STRs de cada tipo
arquivos_str = busca_arquivos('REP1')
dados = []
for idx, arquivo in enumerate(arquivos_str):
  print(f'> lido: {arquivo}')
  # print(arquivo)
  result_csv = pd.read_csv(arquivo, delimiter = ";")

  total_repeticoes = result_csv['repeticoes'].sum()


  id_arquivo = quebra_nome_arquivo(arquivo)
  # print(f'{id_arquivo[0]} {id_arquivo[2]} {contagem_unidades}')
  dados.append({
    "origem": id_arquivo[0],
    "tipo": id_arquivo[2],
    "quantidade": total_repeticoes
  })
df_quantidades = pd.DataFrame(dados)

df_quantidades["tipo"] = df_quantidades["tipo"].astype(int)

df_pivot = df_quantidades.pivot_table(
    index=["tipo"],
    columns="origem",
    values="quantidade",
    aggfunc="sum",
    fill_value=0
).reset_index()
df_pivot.columns.name = None

df_pivot = df_pivot.sort_values(by='tipo')

url_arquivo = pasta_analise + 'SOMA_1.csv'
df_pivot.to_csv(url_arquivo, sep=';', index=False)
print(f'  > gravado: {url_arquivo}')


print('>>> fim')

In [ ]:
# PASSO 3.1.1 - visualizacao dos arquivos gerados no PASSO 3.1
# Gera Tabela 2.a - resultados e discussão

arquivo_soma = pasta_analise + 'SOMA_1.csv'
result_csv = pd.read_csv(arquivo_soma, delimiter = ";")

numero_para_nome_map = {v: k for k, v in tipo_nucleotideo_map.items()}
result_csv.insert(
    0,
    "Tipo de STR",
    result_csv["tipo"].map(numero_para_nome_map)
)

mapa_colunas = df_amostras.set_index("id")["Cepa (strain)"].to_dict()
result_csv = result_csv.rename(columns=mapa_colunas)

result_csv = result_csv.drop(columns='tipo')

url_arquivo = pasta_analise + 'TABELA_2A.csv'
result_csv.to_csv(url_arquivo, sep=';', index=False)
print(f'  > gravado: {url_arquivo}')

result_csv

In [ ]:
# PASSO 3.2 - agrupamento dos arquivos gerados no PASSO 3 -  gera arquivoo SOMA_2
#   - quantidade de STRs localizados para cada unidade e tipo de STR (tamanho) de cada amostra
# Tabela 2.b - resultados e discussão

# REP2 => total geral de cada str, tipo e unidade
import pandas as pd

arquivos_str = busca_arquivos('REP2')
dados = []
for idx, arquivo in enumerate(arquivos_str):
  result_csv = pd.read_csv(arquivo, delimiter = ";")
  dados.append(result_csv)
df_concatenado = pd.concat(dados, ignore_index=True)

df_concatenado["tipo"] = df_concatenado["tipo"].astype(int)
df_pivot = df_concatenado.pivot_table(
    index=["tipo", "unidade"],
    columns="origem",
    values="repeticoes",
    aggfunc="sum",
    fill_value=0
).reset_index()
df_pivot.columns.name = None

# df_pivot = df_pivot.sort_values(by=['unidade', 'tipo'])
df_pivot = df_pivot.sort_values(by='tipo')
df_pivot = df_pivot.reset_index()

url_arquivo = pasta_analise + 'SOMA_2.csv'
df_pivot.to_csv(url_arquivo, sep=';', index=False)
print(f'  > gravado: {url_arquivo}')

df_pivot

In [ ]:
# PASSO 3.2.1 - visualizacao dos arquivos gerados no PASSO 3.2
#   - quantidade de STRs localizados para cada unidade e tipo de STR (tamanho) de cada amostra
# Gera Tabela 2.b - resultados e discussão

arquivo_soma = pasta_analise + 'SOMA_2.csv'
result_csv = pd.read_csv(arquivo_soma, delimiter = ";")

numero_para_nome_map = {v: k for k, v in tipo_nucleotideo_map.items()}
result_csv.insert(
    0,
    "Tipo de STR",
    result_csv["tipo"].map(numero_para_nome_map)
)

mapa_colunas = df_amostras.set_index("id")["Cepa (strain)"].to_dict()
result_csv = result_csv.rename(columns=mapa_colunas)

result_csv = result_csv.drop(columns=['index', 'tipo'])

url_arquivo = pasta_analise + 'TABELA_2B.csv'
result_csv.to_csv(url_arquivo, sep=';', index=False)
print(f'  > gravado: {url_arquivo}')

result_csv

In [ ]:
# PASSO 3.3 - agrupamento dos arquivos gerados no PASSO 3 - gera arquivo SOMA_3
#   - quantidade de STRs localizados para cada unidade, copias e tipo de STR (tamanho) de cada amostra
#   - quantidade de STRs localizados para cada unidade e tipo de STR (tamanho) de cada amostra
# Tabela 2.c - resultados e discussão

# REP3 => total geral de cada str, tipo e unidade e copias
import pandas as pd

arquivos_str = busca_arquivos('REP3')
dados = []
for idx, arquivo in enumerate(arquivos_str):
  result_csv = pd.read_csv(arquivo, delimiter = ";")
  dados.append(result_csv)
df_concatenado = pd.concat(dados, ignore_index=True)

df_concatenado["tipo"] = df_concatenado["tipo"].astype(int)
df_pivot = df_concatenado.pivot_table(
    index=["tipo", "unidade", "copias"],
    columns="origem",
    values="repeticoes",
    aggfunc="sum",
    fill_value=0
).reset_index()
df_pivot.columns.name = None

# df_pivot = df_pivot.sort_values(by=['unidade', 'tipo'])
df_pivot = df_pivot.sort_values(by=['tipo', 'copias'])
df_pivot = df_pivot.reset_index()

url_arquivo = pasta_analise + 'SOMA_3.csv'
df_pivot.to_csv(url_arquivo, sep=';', index=False)
print(f'  > gravado: {url_arquivo}')

df_pivot

In [ ]:
# PASSO 3.3.1 - visualizacao dos arquivos gerados no PASSO 3.3
# aki - esse passo eh igual para 3.1.1 e 3.2.1 - criar funcao separada
#   - quantidade de STRs localizados para cada unidade, tipo e numero de cópias  de STR (tamanho) de cada amostra
# Gera Tabela 2.c - resultados e discussão
# 14-09-2025

arquivo_soma = pasta_analise + 'SOMA_3.csv'
result_csv = pd.read_csv(arquivo_soma, delimiter = ";")

numero_para_nome_map = {v: k for k, v in tipo_nucleotideo_map.items()}
result_csv.insert(
    0,
    "Tipo de STR",
    result_csv["tipo"].map(numero_para_nome_map)
)

mapa_colunas = df_amostras.set_index("id")["Cepa (strain)"].to_dict()
result_csv = result_csv.rename(columns=mapa_colunas)

result_csv = result_csv.drop(columns=['index', 'tipo'])

url_arquivo = pasta_analise + 'TABELA_2C.csv'
result_csv.to_csv(url_arquivo, sep=';', index=False)
print(f'  > gravado: {url_arquivo}')

result_csv

In [ ]:
# aki aki - teste diagrama
import pandas as pd
import matplotlib.pyplot as plt
from upsetplot import UpSet, from_memberships

# Caminho do arquivo
url_arquivo = pasta_analise + 'SOMA_3.csv'

# Carregar os dados com separador correto
df = pd.read_csv(url_arquivo, sep=";")

# Lista das amostras
amostras = ['CBS 554.65', 'CBS 513.88', 'KJC3', 'KYF3', 'CCTCC 206047']

# Função para obter o conjunto de amostras onde o STR está presente
def obter_conjunto(row):
    return tuple([amostra for amostra in amostras if row[amostra] > 0])

# Aplicar a função para criar a coluna de conjuntos
df['conjunto'] = df.apply(obter_conjunto, axis=1)

# Gerar gráfico para cada tipo de STR
tipos_str = df['Tipo de STR'].unique()

for tipo in tipos_str:
    df_tipo = df[df['Tipo de STR'] == tipo]

    # Gerar Series com conjuntos
    series = from_memberships(df_tipo['conjunto'], data=1)

    # Agrupar conjuntos duplicados
    series_agrupada = series.groupby(series.index).sum()

    # Plotar UpSetPlot
    plt.figure(figsize=(12, 6))
    upset = UpSet(series_agrupada, show_counts=True, sort_by='degree')
    upset.plot()
    plt.suptitle(f"Interseções de STRs - Tipo {tipo}")
    plt.tight_layout()
    plt.show()

In [ ]:
# PASSO 4 - busca por STRs de mesmo tamanho (copias) que aparecem em TODAS as amostras
# 14-09-2025

arquivo_soma = pasta_analise + 'SOMA_3.csv'
result_csv = pd.read_csv(arquivo_soma, delimiter = ";")

# montar a query dinamicamente
# ASM285v2 > 0 & ASM1928827v1 > 0 & ASM2978390v1 > 0 & ASM2978392v1 > 0 & ASM4765177v1 > 0
labels_repeticao = [label for label in labels_amostras] # levanta as colunas com mesmo nome das amostras q é a coluna que possui o numero de repeticoes
query_str = " & ".join([f"{col} > 0" for col in labels_repeticao])
print(f' > lido: {query_str}')

df_result_set = pd.DataFrame(result_csv.query(query_str))
df_result_set.reset_index(drop=True, inplace=True)
df_result_set = df_result_set.drop(columns=['index'])

url_arquivo = pasta_analise + 'INCLUSIVOS.csv'
df_result_set.to_csv(url_arquivo, sep=';', index=False)

print(f' > gravado: {url_arquivo}')

In [ ]:
AKI - Diagrama de Venn

# PASSO 4.1 Diagrama de Venn - STRs de mesmo tamanho (copias) que aparecem em
#   TODAS as amostras

import matplotlib.pyplot as plt
from matplotlib_venn import venn3

def diagrama_veen(id_analise, only_1928827, only_2978390, only_2978392, inter_1928827_2978390, inter_1928827_2978392, inter_2978390_2978392, inter_todos):
  plt.figure(figsize=(14, 6))
  venn = venn3(subsets=(only_1928827, only_2978390, inter_1928827_2978390,
                        only_2978392, inter_1928827_2978392, inter_2978390_2978392, inter_todos),
              set_labels=(labels_amostras))

  legend_labels = [
      f"Somente ASM1928827v1: {only_1928827}",
      f"Somente ASM2978390v1: {only_2978390}",
      f"Somente ASM2978392v1: {only_2978392}",
      f"ASM1928827v1 & ASM2978390v1: {inter_1928827_2978390}",
      f"ASM1928827v1 & ASM2978392v1: {inter_1928827_2978392}",
      f"ASM2978390v1 & ASM2978392v1: {inter_2978390_2978392}",
      f"Todos: {inter_todos}"
  ]
  plt.legend(legend_labels, loc="lower left",  bbox_to_anchor=(1, 0.5), fontsize=10)
  # plt.legend(legend_labels, loc="center left", bbox_to_anchor=(0.65, 0.5), fontsize=10)
  # 'best', 'upper right', 'upper left', 'lower left', 'lower right', 'right', 'center left', 'center right', 'lower center', 'upper center', 'center'

  plt.title("Distribuição dos STR-" + id_analise + " nas amostras")

  # salvar aquivo de imagem
  arquivo_png = pasta_analise + "veen_" + id_analise + ".png"
  if os.path.exists(arquivo_png):
    os.remove(arquivo_png)
  plt.savefig(arquivo_png)

  plt.show()


def diagrama_veen_unweighted(id_analise, only_1928827, only_2978390, only_2978392, inter_1928827_2978390, inter_1928827_2978392, inter_2978390_2978392, inter_todos):
  plt.figure(figsize=(6, 6))
  # Desenha o diagrama unweighted: todos os subsets recebem "1"
  venn = venn3(subsets=(1, 1, 1, 1, 1, 1, 1),
            set_labels=(labels_amostras))

  # Atualiza os labels para exibir os valores reais
  if venn.get_label_by_id('100'):
      venn.get_label_by_id('100').set_text(str(only_1928827))
  if venn.get_label_by_id('010'):
      venn.get_label_by_id('010').set_text(str(only_2978390))
  if venn.get_label_by_id('001'):
      venn.get_label_by_id('001').set_text(str(only_2978392))
  if venn.get_label_by_id('110'):
      venn.get_label_by_id('110').set_text(str(inter_1928827_2978390))
  if venn.get_label_by_id('101'):
      venn.get_label_by_id('101').set_text(str(inter_1928827_2978392))
  if venn.get_label_by_id('011'):
      venn.get_label_by_id('011').set_text(str(inter_2978390_2978392))
  if venn.get_label_by_id('111'):
      venn.get_label_by_id('111').set_text(str(inter_todos))

  # legend_labels = [
  #     f"Somente ASM1928827v1: {only_1928827}",
  #     f"Somente ASM2978390v1: {only_2978390}",
  #     f"Somente ASM2978392v1: {only_2978392}",
  #     f"ASM1928827v1 & ASM2978390v1: {inter_1928827_2978390}",
  #     f"ASM1928827v1 & ASM2978392v1: {inter_1928827_2978392}",
  #     f"ASM2978390v1 & ASM2978392v1: {inter_2978390_2978392}",
  #     f"Todos: {inter_todos}"
  # ]
  # plt.legend(legend_labels, loc="lower left",  bbox_to_anchor=(1, 0.5), fontsize=10)

  plt.title("Distribuição dos STR-" + id_analise + " nas amostras")

  # salvar aquivo de imagem
  arquivo_png = pasta_analise + "veen_unweighted_" + id_analise + ".png"
  if os.path.exists(arquivo_png):
    os.remove(arquivo_png)
  plt.savefig(arquivo_png)

  plt.show()

# diagrama_veen("'teste diagrama com tamanhos proporcionais aas areas dos conjuntos de valores'", 5, 5, 5, 10, 10, 10, 20)
# diagrama_veen_unweighted("'teste diagrama com tamanhos iguais (nao ponderado - unweighte)'", 5, 5, 5, 10, 10, 10, 20)

arquivos_analise = busca_arquivos('XXX-DET')

for idx, arquivo in enumerate(arquivos_analise):
  result_analise = pd.read_csv(arquivo, delimiter = ";")

  # Calcular interseções
  df_1928827 = result_analise.query('repeticoes_ASM1928827v1  > 0 & repeticoes_ASM2978390v1 == 0 & repeticoes_ASM2978392v1 == 0')
  df_2978390 = result_analise.query('repeticoes_ASM1928827v1 == 0 & repeticoes_ASM2978390v1  > 0 & repeticoes_ASM2978392v1 == 0')
  df_2978392 = result_analise.query('repeticoes_ASM1928827v1 == 0 & repeticoes_ASM2978390v1 == 0 & repeticoes_ASM2978392v1  > 0')
  only_1928827 = len(df_1928827)
  only_2978390 = len(df_2978390)
  only_2978392 = len(df_2978392)

  df_1928827_2978390 = result_analise.query('repeticoes_ASM1928827v1  > 0 & repeticoes_ASM2978390v1  > 0 & repeticoes_ASM2978392v1 == 0')
  df_1928827_2978392 = result_analise.query('repeticoes_ASM1928827v1  > 0 & repeticoes_ASM2978390v1 == 0 & repeticoes_ASM2978392v1  > 0')
  df_2978390_2978392 = result_analise.query('repeticoes_ASM1928827v1 == 0 & repeticoes_ASM2978390v1  > 0 & repeticoes_ASM2978392v1  > 0')
  inter_1928827_2978390 = len(df_1928827_2978390)
  inter_1928827_2978392 = len(df_1928827_2978392)
  inter_2978390_2978392 = len(df_2978390_2978392)

  df_todos = result_analise.query('repeticoes_ASM1928827v1 > 0 & repeticoes_ASM2978390v1 > 0 & repeticoes_ASM2978392v1 > 0')
  inter_todos = len(df_todos)

  id_arquivo = quebra_nome_arquivo(arquivo)
  diagrama_veen_unweighted(id_arquivo[2], only_1928827, only_2978390, only_2978392, inter_1928827_2978390, inter_1928827_2978392, inter_2978390_2978392, inter_todos)

In [ ]:
# PASSO 5 - busca por STRs de mesmo tamanho (copias) que aparecem somente
#   em uma das amostras - monta arquivo _EXCLUSIVOS
# 14-09-2025 09-10-2025

arquivo_soma = pasta_analise + 'SOMA_3.csv'
result_csv = pd.read_csv(arquivo_soma, delimiter = ";")

# dados = []
# ASM285v2 > 0 & ASM1928827v1 == 0 & ASM2978390v1 == 0 & ASM2978392v1 == 0 & ASM4765177v1 == 0
labels_repeticao = [label for label in labels_amostras] # levanta as colunas com mesmo nome das amostras q é a coluna que possui o numero de repeticoes
for label_amosta in labels_repeticao:
    # montar a query dinamicamente cada query seta a coluna com nome da amostras > 0 e as demais == 0
    # para a coluna atual: > 0
    condicoes = [f"{c} > 0" if c == label_amosta else f"{c} == 0" for c in labels_repeticao]

    # montar a query
    query_str = " & ".join(condicoes)
    print(f' > lido: {query_str}')
    # print(f"Arquivo: {arquivo} | Coluna positiva: {col} | Linhas encontradas: {len(dt)}")

    # aplicar filtro
    df_result_set = result_csv.query(query_str)

    df_result_set = pd.DataFrame(result_csv.query(query_str))
    df_result_set.reset_index(drop=True, inplace=True)
    df_result_set = df_result_set.drop(columns=['index'])

    url_arquivo = f'{pasta_analise}{label_amosta}_EXCLUSIVOS.csv'
    df_result_set.to_csv(url_arquivo, sep=';', index=False)
    print(f' > gravado: {url_arquivo} ({len(url_arquivo)})')
    # dados.append(df_result_set)

# df_concatenado = pd.concat(dados, ignore_index=True)

# url_arquivo_fim = pasta_analise + 'GERAL_EXCLUSIVOS.csv'
# df_concatenado.to_csv(url_arquivo_fim, sep=';', index=False)
# dados.append(df_result_set)
# print(f'> fim > gravado: {url_arquivo_fim} (len({url_arquivo_fim})')

In [ ]:
# PASSO 6 - Busca origem dos STRs exclusivos de cada amostra
#  - ler arquivos EXCLUSIVOS de cada genoma e buscar as posições dos STRs
#     nos arquivos de STRs levantados sem duplicidades (_TIPO_)
#     -> gera arquivos SELECT com as informações de loci -> uma linha para cada
#        repetição (sequencial_loci) dos STRs exclusivos de cada cepa

import pandas as pd
import os

def ler_arquivo_str(nome_amostra, tipo):
  dados = pd.DataFrame()
  arquivo = f'{pasta_analise}{nome_amostra}_TIPO_{str(tipo)}.csv'
  dados = pd.read_csv(arquivo, delimiter = ";")
  print(f'   > lido str (TIPO): {arquivo} ({len(dados)})')
  return dados

tipo_atual = ''
id_amostra_atual = ''
dados_str = pd.DataFrame()

erros = []

arquivos_exclusivos = busca_arquivos('EXCLUSIVOS')
for idx, arquivo in enumerate(arquivos_exclusivos):
  nome_arquivo = os.path.basename(arquivo) # Obter apenas o nome do arquivo sem o diretório
  parte_principal = os.path.splitext(nome_arquivo)[0] # Remover a extensão do arquivo
  partes = parte_principal.split('_')
  id_amostra = partes[0]

  dados_exclusivos = pd.read_csv(arquivo, delimiter = ";")
  print(f'   > lido exclusivos: {arquivo} ({len(dados_exclusivos)})')
  dados_exclusivos = dados_exclusivos.sort_values(['tipo', 'unidade', 'copias'], ascending=[True, True, True])

  linhas_novas = []
  for _, row in dados_exclusivos.iterrows():
    if id_amostra != id_amostra_atual or tipo_atual != row['tipo']:
      dados_str = ler_arquivo_str(id_amostra, row['tipo'])
    tipo_atual = row['tipo']
    id_amostra_atual = id_amostra

    # id_amostra
    tipo = row['tipo']
    unidade = row['unidade']
    copias = row['copias']

    df_str_localizados = dados_str.query("`unidade` == @unidade & copias == @copias")
    df_str_localizados = pd.DataFrame()
    try:
        df_str_localizados = dados_str.query("`unidade` == @unidade & copias == @copias")
    except Exception as e:
        print(f'Erro ao buscar STR: {e}')
        erros.append(f'ERROR {id_amostra} Erro ao buscar STR: {e} -> row: {row}')
        continue
    if df_str_localizados.empty:
        erros.append(f'ERROR {id_amostra} df_str_localizados.empty -> row: {row}')
        print(f'      > Nenhum STR localizado para unidade={unidade}, cópias={copias}')
    repeticoes_na_amostra = int(row[f'{id_amostra}'])
    if repeticoes_na_amostra != len(df_str_localizados): # quantidade de repeticoes da amostra
        erros.append(f'ERROR {id_amostra} quantidade de repeticoes direfentes -> row: {row}')

    sequencial_loci = 0
    for ix_str_localizado, str_localizado in df_str_localizados.iterrows():
      nova_linha = {}
      nova_linha['origem'] = id_amostra
      nova_linha['tipo'] = tipo
      nova_linha['unidade'] = unidade
      nova_linha['copias'] = copias
      nova_linha['inicio_loci'] = str_localizado['inicio_loci']
      nova_linha['fim_unidade'] = str_localizado['fim_unidade']
      nova_linha['fim_loci'] = str_localizado['fim_loci']
      nova_linha['repeticoes_na_amostra'] = repeticoes_na_amostra
      nova_linha['sequencial_loci'] = sequencial_loci
      sequencial_loci = sequencial_loci + 1
      # print(f'      > NOVA_LINHA: \n{nova_linha})')
      linhas_novas.append(nova_linha)

  df_resultado = pd.DataFrame(linhas_novas)
  arquivo_saida = f'{pasta_analise}{id_amostra}_SELECT.csv'
  df_resultado.to_csv(arquivo_saida, sep=';', index=False)
  print(f'   > gravado: {arquivo_saida} ({len(df_resultado)})')

print(f'>>> fim - erros: {len(erros)}')
if len(erros) > 0:
  print(f'!!! erros: {erros}')

In [ ]:
# PASSO 7 - levantar as regioes flanqueadoras dos STRs exclusivos - gera arquivos _FLANK_
#   ler arquivos SELECT e buscar regiões flanqueadoras
# 14-09-2025 15-09-2025 10-10-2025
# Lê arquivos de STRs marcados como SELECT.
# Identifica a origem genômica e o tipo de STR.
# Busca a sequência FASTA correspondente à origem.
# Para cada STR:
#     Calcula os flancos (create_flank) com base nas posições.
#     Adiciona informações como fragmento, montante, jusante, bordas etc.
# Salva os dados enriquecidos em um novo arquivo com sufixo FLANK.

!pip install biopython

import pandas as pd
from str_utils import busca_arquivos, busca_sequencia_fasta, create_flank, gravar_arquivo_saida

arquivos_str = busca_arquivos('SELECT')
origem_atual = ''
sequencia = ''
quantidade_arquivos_lidos = 0
quantidade_arquivos_gravados = 0

for idx, arquivo in enumerate(arquivos_str):
  df_arq_str = pd.read_csv(arquivo, delimiter = ";")
  print(f' > lido: {arquivo} ({len(df_arq_str)})')
  quantidade_arquivos_lidos = quantidade_arquivos_lidos + 1

  linhas_atualizadas = []
  for idx_unidade_str, unidade_str in df_arq_str.iterrows():
    origem = unidade_str['origem']
    if origem != origem_atual:
        sequencia = busca_sequencia_fasta(origem)
    origem_atual = origem

    # print(f"processando{idx_unidade_str + 1} de ({len(df_arq_str)})...")
    flank = create_flank(sequencia, unidade_str['inicio_loci'], unidade_str['fim_loci'])
    linha_completa = unidade_str.to_dict()
    linha_completa['inicio_amostral'] = flank['inicio_amostral']
    linha_completa['fim_amostral'] = flank['fim_amostral']
    linha_completa['fragment'] = flank['fragment']
    linha_completa['montante'] = flank['montante']
    linha_completa['jusante'] = flank['jusante']
    linha_completa['full_flank'] = flank['full_flank']
    linha_completa['borda_esquerda'] = flank['borda_esquerda']
    linha_completa['borda_direita'] = flank['borda_direita']
    linhas_atualizadas.append(linha_completa)

  df_flankeado = pd.DataFrame(linhas_atualizadas)
  arq_sel = f'{pasta_analise}{origem}_FLANK.csv'
  df_flankeado.to_csv(arq_sel, sep=';', index=False)
  quantidade_arquivos_gravados = quantidade_arquivos_gravados + 1
  print(f' > gravado: {arq_sel} ({len(df_flankeado)})')

print(f'>>> fim: {quantidade_arquivos_lidos} arquivos lidos e {quantidade_arquivos_gravados} gravados')

In [ ]:
# Passo XX verificar se áreas levantadas coincidem na amostra
# !pip install biopython
from str_utils import busca_sequencia_fasta

url_arquivo =  f'{pasta_analise}FIM_full.csv'
result = pd.read_csv(url_arquivo, delimiter = ";")
print(f"> leitura: {url_arquivo} ({len(result)})")

origem_fasta = ''
sequencia_fasta = ''
quant_erros = 0
erros = []
for idx, str_loc in result.iterrows():
  print(f"...analisando {idx + 1} de {len(result)}")
  if origem_fasta != str_loc['origem']:
    sequencia_fasta = busca_sequencia_fasta(str_loc['origem'])
    print(f"> fasta..: {sequencia_fasta[:100]}")
  origem_fasta = str_loc['origem']

  try:
    area_loc = ( str_loc['borda_esquerda'] +
      str_loc['montante'] +
      str_loc['fragment'] +
      str_loc['jusante'] +
      str_loc['borda_direita'] )
  except Exception as e:
      erros.append(f'ERRO exceção - cálculo de area_loc - inesperada: {e} str_loc: {str_loc}')

  # print(area_total)
  inicio_amostra = str_loc['inicio_amostral']
  fim_amostra    = inicio_amostra + len(area_loc)

  try:
    area_original = sequencia_fasta[inicio_amostra:fim_amostra]
  except IndexError as e:
      erros.append(f'ERRO exceção de índice: {e} str_loc: {str_loc}')
  except Exception as e:
      erros.append(f'ERRO erro ao buscar sequencia fasta: {e} str_loc: {str_loc}')

  if area_loc != area_original:
    erros.append(f'ERRO área da amostra é diferente da área localizada no genoma original - original.: {area_original} analisada: {area_loc} str_loc: {str_loc}')

print(f'>>> fim - erros: {len(erros)}')
if len(erros) > 0:
  for erro in erros:
    print(f'!!! erro: \n        {erro}')


In [ ]:
# PASSO 7.1 - visualizar STR e áreas flankeadas
# 15-09-2025 fragm

import matplotlib.pyplot as plt
import random
import matplotlib.ticker as ticker

import pandas as pd
# from str_utils import busca_arquivos, busca_sequencia_fasta, create_flank, gravar_arquivo_saida

dados = []
arquivos_flank = busca_arquivos('FLANK')
for idx, arquivo in enumerate(arquivos_flank):
  df_arq_flank = pd.read_csv(arquivo, delimiter = ";")
  print(f' > lido: {arquivo} ({len(df_arq_flank)})')
  # id_arquivo = quebra_nome_arquivo(arquivo)
  # origem = id_arquivo[0]
  # tipo = id_arquivo[2]
  dados.append(df_arq_flank)
df_flankeados = pd.concat(dados, ignore_index=True)
# url_arquivo = pasta_analise + 'FLANQUEAMENTOS.csv'
# df_flankeados.to_csv(url_arquivo, sep=';', index=False)
# print(f'>>> fim > gravado: {url_arquivo}')


# # Função para gerar sequência aleatória de DNA
# def random_dna(length):
#     return ''.join(random.choices(['A','T','G','C'], k=length))
# Criando regiões do exemplo
# seq_before = random_dna(200)       # região livre antes da montante
# montante = random_dna(150)         # região montante
# str_seq = "GA" * 50                 # STR central (100 bp de GA)
# jusante = random_dna(150)           # região jusante
# seq_after = random_dna(200)         # região livre depois da jusante

##################################
# query_res = df_flankeados.query('(tipo * copias) > 30')
# flank_sel = query_res.iloc[int(len(query_res)/ 2)]
flank_sel = df_flankeados.iloc[0]
##################################

seq_before = flank_sel['borda_esquerda']
montante = flank_sel['montante']
str_seq = flank_sel['fragment']
jusante = flank_sel['jusante']
seq_after = flank_sel['borda_direita']
unidade_str = flank_sel['unidade']
tipo_str = flank_sel['tipo']
offset = flank_sel['inicio_amostral']
tipo = flank_sel['tipo']
copias = flank_sel['copias']

#  > fragment:	Sequência entre inicio e fim
#  > forward_flank:	Região anterior ao fragmento, com tamanho mínimo de 200
#  > reverse_flank:	Região posterior ao fragmento, com tamanho mínimo de 200
#  > full_flank:	Flanqueadoras + fragmento central
#  > borda_esquerda:	Região antes da flanqueadora esquerda (200 bases)
#  > borda_direita:	Região após a flanqueadora direita (200 bases)
#  > inicio_amostral:	Índice inicial da sequência total estendida
#  > fim_amostral:	Índice final da sequência total estendida

# Concatenando para formar a sequência final
seq = seq_before + montante + str_seq + jusante + seq_after

# Definir posições de cada região
montante_start = len(seq_before)
montante_end = montante_start + len(montante)
str_start = montante_end
str_end = str_start + len(str_seq)
jusante_start = str_end
jusante_end = jusante_start + len(jusante)
borda_direita_end = jusante_end + len(seq_after)

# Mapear bases para números
bases = {'A':1, 'T':2, 'G':3, 'C':4}

# Cores para cada base
base_colors = {'A':'red', 'T':'blue', 'G':'green', 'C':'orange'}

# Criar stem plot
fig, ax = plt.subplots(figsize=(25,5))

# Fundo das regiões com transparência menor
ax.axvspan(0, montante_start, color='lightgray', alpha=0.2)
ax.axvspan(montante_start, montante_end, color='skyblue', alpha=0.2)
ax.axvspan(str_start, str_end, color='gold', alpha=0.2)
ax.axvspan(jusante_start, jusante_end, color='lightgreen', alpha=0.2)
ax.axvspan(jusante_end, len(seq), color='lightgray', alpha=0.2)

# Linhas verticais pontilhadas delimitando regiões
for pos in [montante_start, montante_end, str_end, jusante_end]:
    ax.axvline(x=pos, color='black', linestyle='--', linewidth=1)

# Stem plot por base (linhas verticais)
for i, base in enumerate(seq):
    if base in bases:
        ax.stem([i+1], [bases[base]],
                linefmt=base_colors[base],
                markerfmt='',
                basefmt=" ")

# Labels das regiões no topo
ax.text(montante_start + len(montante)/2, 4.5, f"Montante ({len(montante)})", ha='center', fontsize=12, fontweight='bold')
ax.text(str_start + len(str_seq)/2, 4.5, f'STR-{tipo_str}: {unidade_str} ({len(str_seq)})', ha='center', fontsize=12, fontweight='bold')
ax.text(jusante_start + len(jusante)/2, 4.5, f"Jusante ({len(jusante)})", ha='center', fontsize=12, fontweight='bold')

# Ajustes visuais
ax.set_ylim(0,5)
ax.set_yticks([1,2,3,4])
ax.set_yticklabels(['A','T','G','C'])

ax.set_xlabel("Posição na sequência")
# ax.set_title("Sequência de DNA com STR e regiões flanqueadoras")

ax.set_xticks([0, montante_start, montante_end, str_start, str_end, jusante_start, jusante_end, borda_direita_end])
# Mantém o gráfico como está, mas altera apenas os labels do eixo x

xticks = ax.get_xticks()           # pega os ticks atuais
# ax.set_xticklabels([int(tick + offset) for tick in xticks])
ax.set_xticklabels([f"{int(tick + offset):,}".replace(",", ".") for tick in xticks])

ax.set_xlabel(f"Posição das porções destacadas na sequência FASTA original da amostra {origem}", labelpad=5, fontsize=12, fontweight='bold')
ax.tick_params(axis='x', labelsize=14, width=2, length=8)
ax.set_ylabel(f"Nucleotídeos", labelpad=5, fontsize=12, fontweight='bold')
ax.tick_params(axis='y', labelsize=14, width=2, length=8)


# Legenda manual
from matplotlib.patches import Patch
# legend_areas = [Patch(facecolor='skyblue', label='Montante', alpha=0.2),
#                    Patch(facecolor='gold', label='STR', alpha=0.2),
#                    Patch(facecolor='lightgreen', label='Jusante', alpha=0.2),
#                    Patch(facecolor='lightgray', label='Região livre', alpha=0.2)]
# ax.legend(handles=legend_areas, title="Áreas", loc='upper right')

base_colors = {'A': 'red', 'C': 'orange', 'T': 'blue', 'G': 'green'}
legend_bases = [Patch(color=base_colors[base], label=base) for base in ['A', 'T', 'G', 'C']]
# legend_total = legend_bases + legend_areas
ax.legend(handles=legend_bases , title="Bases", loc='upper left')

# ax.set_title(f'{str_seq} tipo: {tipo} copias: {copias}')

# fig.text(0.5, -0.05, f'Montante: {montante}', fontsize=12)
# fig.text(0.5, -0.05, f'Jusante: {jusante}', ha='center', fontsize=12)

# salvar aquivo de imagem
arquivo_png = "/content/drive/MyDrive/pesquisa/regioes_flanqueadoras.png"
plt.savefig(arquivo_png, bbox_inches='tight', pad_inches=0, dpi=300)

plt.show()

In [ ]:
# PASSO 8 - Montar os primers com primer3 - gera arquivos PRIMERS
# 15-09-2025 10-10-2025
!pip install primer3-py

import pandas as pd
import re
# from str_utils import busca_arquivos, busca_sequencia_fasta, create_flank, gravar_arquivo_saida
import primer3


def safe_len(val):
    if pd.isna(val):   # se for NaN
        return 0
    return len(str(val))


# Parâmetros de desenho dos primers
parametros_primers = {
    'PRIMER_MIN_SIZE': 18,
    'PRIMER_OPT_SIZE': 20,
    'PRIMER_MAX_SIZE': 23,
    'PRIMER_MIN_TM': 57.0,
    'PRIMER_OPT_TM': 59.0,
    'PRIMER_MAX_TM': 62.0,
    'PRIMER_MAX_POLY_X': 5,
    'PRIMER_PAIR_MAX_DIFF_TM': 5.0,
}

def sequencia_valida(seq):
    return bool(re.fullmatch(r'[ACGTNS]+', seq))

def limpar_sequencia(seq):
    return re.sub(r'[^ACGTN]', 'N', seq)

# Função para desenhar primers para uma linha

def desenhar_primers(row):
    id = f"{row['origem']}-{row['unidade']}-{row['copias']}"
    seq = str(row['full_flank']).upper()
    target_start = safe_len(row['montante'])
    target_end   = safe_len(row['fragment'])
    if not sequencia_valida(seq):
        print(f">>>>>>>>>>>> PRIMER_ERROR: Sequência inválida: {seq}")
        return pd.Series({'PRIMER_ERROR': 'Sequência inválida'})
    entrada = {
      'SEQUENCE_ID': id,
      'SEQUENCE_TEMPLATE': seq,
      'SEQUENCE_TARGET': [target_start, target_end]
    }
    resultado = primer3.bindings.design_primers(entrada, parametros_primers)
    return pd.Series(resultado)


quantidade_arquivos_lidos = 0
quantidade_arquivos_gravados = 0

arquivos_flank = busca_arquivos('FLANK')
for idx, arquivo in enumerate(arquivos_flank):
  df_arq_flank = pd.read_csv(arquivo, delimiter = ";")
  print(f' > lido: {arquivo} ({len(df_arq_flank)})')
  quantidade_arquivos_lidos = quantidade_arquivos_lidos + 1

  df_arq_flank['full_flank'] = df_arq_flank['full_flank'].apply(limpar_sequencia)

  nome_arquivo = os.path.basename(arquivo) # Obter apenas o nome do arquivo sem o diretório
  parte_principal = os.path.splitext(nome_arquivo)[0] # Remover a extensão do arquivo
  partes = parte_principal.split('_')
  id_amostra = partes[0]

  # Aplicar a função para gerar os primers
  df_resultado_primers = df_arq_flank.apply(desenhar_primers, axis=1)
  # juntar os resultados ao DataFrame original
  df_primer = df_arq_flank.join(df_resultado_primers)

  arq_primer = f'{pasta_analise}{id_amostra}_PRIMER.csv'
  df_primer.to_csv(arq_primer, sep=';', index=False)
  print(f' > gravado: {arq_primer} ({len(df_primer)})')
  quantidade_arquivos_gravados = quantidade_arquivos_gravados + 1

print(f'>>> fim: {quantidade_arquivos_lidos} arquivos lidos e {quantidade_arquivos_gravados} gravados')

In [ ]:
arquivos_primer = busca_arquivos('FLANK')

for idx, arquivo in enumerate(arquivos_primer):
  # df_arq_primer = pd.read_csv(arquivo, delimiter = ";")
  # print(f' > lido: {arquivo} ({len(df_arq_primer)})')
  # df_arq_primer['entropy'] = df_arq_primer['unidade'].apply(shannon_entropy)
  # print(df_arq_primer['entropy'])
  print(arquivo)
print('fim')


In [ ]:
# PASSO 9 - Montar os Amplicons - gera arquivos _AMP_
#   avalia os 5 primers retornados para cada full_flank (flank_L _ STR + flank_R)
#     procura melhor alinhamento par ao primer
# 15-09-2025 16-09-2025 17-09-2025 amplicon

# aKI Aki duvida sobre a montagem (localizacao do primer reverse)
# busca_melhor_alinhamento
# a distrancia de hamer esta ficando alta

from str_utils import localiza_primers, busca_melhor_alinhamento

dados = []
arquivos_primer = busca_arquivos('PRIMER')

for idx, arquivo in enumerate(arquivos_primer):
  df_arq_primer = pd.read_csv(arquivo, delimiter = ";")
  print(f' > lido: {arquivo} ({len(df_arq_primer)})')
  id_arquivo = quebra_nome_arquivo(arquivo)
  origem = id_arquivo[0]

  registros = []
  for i_primer, primer3_result in df_arq_primer.iterrows():
    # print(f'{arq_primer}')
    # filtra os primers
    primers = []
    vistos = set()

    for i in range(5):
        primer_left = primer3_result.get(f'PRIMER_LEFT_{i}_SEQUENCE', "")
        primer_right = primer3_result.get(f'PRIMER_RIGHT_{i}_SEQUENCE', "")

        val_left = primer3_result.get(f"PRIMER_LEFT_{i}", "")
        val_right = primer3_result.get(f"PRIMER_RIGHT_{i}", "")
        left_pos = -1
        left_len = 0
        right_pos = -1
        right_len = 0
        if pd.notna(val_left) and val_left != "":
            # remove colchetes e espaços
            val_left_str = str(val_left).strip("[] ")
            left_pos, left_len = map(int, val_left_str.split(","))
        else:
            left_pos, left_len = -1, 0

        if pd.notna(val_right) and val_right != "":
            val_right_str = str(val_right).strip("[] ")
            right_pos, right_len = map(int, val_right_str.split(","))
        else:
            right_pos, right_len = -1, 0


        primer_pair_penalty = primer3_result.get(f'PRIMER_PAIR{i}_PENALTY', "")
        primer_left_penalty = primer3_result.get(f'PRIMER_LEFT{i}_PENALTY', "")
        primer_right_penalty = primer3_result.get(f'PRIMER_RIGHT{i}_PENALTY', "")
        par = (primer_left, primer_right)

        montante = primer3_result.get('montante')
        montante = primer3_result.get('montante')
        jusante = primer3_result.get('jusante')
        full_flank = primer3_result.get('full_flank')
        fragment = primer3_result.get('fragment')

        # if par not in vistos and primer_left and primer_right:
        if pd.notna(primer_left) and pd.notna(primer_right) and par not in vistos and primer_left and primer_right and pd.notna(montante) and pd.notna(jusante):
          # procurar melhor alinhamento pois nem sempre será idêntico
          # print(f"primer: {i} montante: {primer3_result.get('montante')} "
          #       f"jusante: {primer3_result.get('jusante')} "
          #       f"i_primer: {i_primer} primer3_result: {primer3_result}")

          inicio_montante = 0
          fim_montante = len(montante)
          # inicio_jusante = fim_montante + len(fragment) - 1 - aki foi ajustado retirando 0 -1
          inicio_jusante = fim_montante + len(fragment)
          fim_jusante = len(full_flank)
          primer_left_start = busca_melhor_alinhamento(full_flank[inicio_montante:fim_montante], primer_left)

          # print(primer_left_start)
          # print(f'jusante: {primer3_result['jusante']}')

          # primer_right_start = busca_melhor_alinhamento(full_flank[inicio_jusante:fim_jusante], primer_right, True)
          primer_right_start = busca_melhor_alinhamento(full_flank[inicio_jusante:fim_jusante], primer_right) # aki testando em inverter e complementar

          print(f'full_flank ({len(full_flank)}): {full_flank}')
          print(f'fragment ({len(fragment)}): {fragment}')
          print(f' > inicio_montante: {inicio_montante}')
          print(f' > fim_montante: {fim_montante}')
          print(f' > inicio_jusante: {inicio_jusante}')
          print(f' > fim_jusante: {fim_jusante}')
          print(f' > primer_left_start: {primer_left_start}')
          print(f' > primer_right_start: {primer_right_start}')
          print(f'   > inicio_jusante: {inicio_jusante}')
          print(f' > primer_left ({len(primer_left)}): {primer_left}')
          print(f' > primer_right ({len(primer_right)}): {primer_right}')
          print(f" > primer_right_recorte ({len(primer_right_start['recorte'])}): {primer_right_start['recorte']}")
          # print(primer_left_start)
          # {'distancia': -1, 'indice': -1, 'recorte': '', 'localizar': ''}

          # amplicon = ''
          # if primer_left_start['distancia'] != -1 and primer_right_start['distancia'] != -1:
          #   primer_esquerdo_extendido = primer3_result['montante'][
          #       primer_left_start['indice']:len(primer3_result['montante'])
          #   ]
          #   primer_direito_extendido = primer3_result['jusante'][
          #       0:primer_right_start['indice'] + len(primer_right)
          #   ]
          #   amplicon = primer_esquerdo_extendido + primer3_result['fragment'] + primer_direito_extendido

          amplicon = ''
          # if primer_left_start['distancia'] != -1 and primer_right_start['distancia'] != -1: # aki - revisar essa captura
              # # Extensão do primer esquerdo
              # amplicon_inicio = primer_left_start['indice']
              # # ampicon_esquerdo = primer3_result['full_flank'][amplicon_inicio:]

              # # Extensão do primer direito
              # incio_primer_right = inicio_jusante + primer_right_start['indice']
              # amplicon_fim = incio_primer_right + len(primer_right)
              # # amplicon_direito = primer3_result['full_flank'][:amplicon_fim]

              # # Construção do amplicon
              # # amplicon = ampicon_esquerdo + primer3_result['fragment'] + amplicon_direito
              # amplicon = primer3_result['full_flank'][amplicon_inicio:amplicon_fim]

          # aki - ultima alteracao, utilizar os indices e tamanho de primer retornado pelo primer3 ao invés de calcular
          if left_pos != -1 and left_len > 0 and right_pos != -1 and right_len > 0:
              # Extensão do primer esquerdo
              amplicon_inicio = left_pos
              # ampicon_esquerdo = primer3_result['full_flank'][amplicon_inicio:]

              # Extensão do primer direito
              incio_primer_right = right_pos
              amplicon_fim = incio_primer_right + right_len
              # amplicon_direito = primer3_result['full_flank'][:amplicon_fim]

              # Construção do amplicon
              # amplicon = ampicon_esquerdo + primer3_result['fragment'] + amplicon_direito
              amplicon = primer3_result['full_flank'][amplicon_inicio:amplicon_fim]


              print(f'    > amplicon_inicio/primer_left_inicio: {amplicon_inicio}')
              print(f'    > incio_primer_right: {incio_primer_right}')
              print(f'    > amplicon_fim: {amplicon_fim}')
              print(f'    > amplicon: {amplicon}')

              primers.append({
                  'primer_left': primer_left,
                  'primer_left_loc': primer_left_start['recorte'],
                  'primer_right': primer_right,
                  'primer_right_loc': primer_right_start['recorte'],
                  'primer_right_loc_direta': primer_right_start['localizar'],
                  'primer_left_distancia': primer_left_start['distancia'],
                  'primer_right_distancia': primer_right_start['distancia'],
                  'primer_pair_penalty': primer_pair_penalty,
                  'primer_left_penalty': primer_left_penalty,
                  'primer_left_index': left_pos,
                  'primer_left_len': left_len,
                  'primer_right_index': right_pos,
                  'primer_right_len': right_len,
                  'primer_right_penalty': primer_right_penalty,
                  'primer_left_start': amplicon_inicio,
                  'primer_right_start': incio_primer_right,
                  'amplicon': amplicon,
                  'full_flank': primer3_result['full_flank'],
                  'montante': primer3_result['montante'],
                  'jusante': primer3_result['jusante']
                  })
              vistos.add(par)

    # print(f'quantidade primers montados: {len(primers)}')

    qt = 0
    for i_p in primers:
      qt = qt + 1
      # print(f'{i_p} - {qt} de {len(primers)}')

      registro_primer = {
          'sequencial': qt,
          'unidade': primer3_result['unidade'],
          'inicio_loci': primer3_result['inicio_loci'],
          'fim_unidade': primer3_result['fim_unidade'],
          'fim_loci': primer3_result['fim_loci'],
          'copias': primer3_result['copias'],
          'tipo': primer3_result['tipo'],
          'origem': primer3_result['origem'],
          'inicio_amostral': primer3_result['inicio_amostral'],
          'fim_amostral': primer3_result['fim_amostral'],
          'fragment': primer3_result['fragment'],
          'montante': primer3_result['montante'],
          'jusante': primer3_result['jusante'],
          'full_flank': primer3_result['full_flank'],
          'borda_esquerda': primer3_result['borda_esquerda'],
          'borda_direita': primer3_result['borda_direita'],

          'primer_left': i_p['primer_left'],
          'primer_left_loc': i_p['primer_left_loc'],
          'primer_left_start': i_p['primer_left_start'],
          'primer_left_distancia': i_p['primer_left_distancia'],
          'primer_right': i_p['primer_right'],
          'primer_right_loc': i_p['primer_right_loc'],
          'primer_right_loc_direta': i_p['primer_right_loc_direta'],
          'primer_right_start': i_p['primer_right_start'],
          'primer_right_distancia': i_p['primer_right_distancia'],
          'primer_pair_penalty': i_p['primer_pair_penalty'],
          'primer_left_penalty': i_p['primer_left_penalty'],
          'primer_right_penalty': i_p['primer_right_penalty'],
          'primer_left_index': i_p['primer_left_index'],
          'primer_left_len': i_p['primer_left_len'],
          'primer_right_index': i_p['primer_right_index'],
          'primer_right_len': i_p['primer_right_len'],
          'amplicon': i_p['amplicon'],
          'tamanho_str': primer3_result['tipo'] * primer3_result['copias'],
          'tamanho_amplicon': len(i_p['amplicon'])
      }
      registros.append(registro_primer)

  # df_registros = pd.concat(registros, ignore_index=True)
  # df_registros = pd.DataFrame([registros])
  # df_registros = pd.DataFrame(registros, columns=colunas_manter)
  df_registros = pd.DataFrame(registros)

  url_arquivo =  f'{pasta_analise}{origem}_AMP.csv'
  df_registros.to_csv(url_arquivo, sep=';', index=False)
  print(f'   > gravado: {url_arquivo} ({len(df_registros)})')

print(f'>>> fim')

In [ ]:
# testes de busca_melhor_alinhamento - antigo mantido
#  ajustar e apagar nao utilizados no utils

from str_utils import busca_melhor_alinhamento
#                       1         2         3         4         5
#              012345678901234567890123456789012345678901234567890
sequencia = "ATGCGTACGTAGCTAGCTAGGATCCGATCGTACGTAGCTAGCTAGCTA"
primer_left  = "ATGCGTACG"   # esperado alinhar no início
primer_right = "TAGCTC"      # esperado alinhar no final
                # TAGCTA GAGCTA GAGCTA
teste = busca_melhor_alinhamento(sequencia, primer_left)
print(teste)

teste2 = busca_melhor_alinhamento(sequencia, primer_right, True)
print(teste2)

In [ ]:
# PASSO 10 - Filtrar amplicons para conferir com BLASTn
#  -> melhores
#  -> gera arquivo FIM (index_amp = sequencial para recuperar resultado do blastN)
#
from datetime import datetime
import pandas as pd
from str_utils import busca_arquivos

# def filtrar_amostras(df):
#   print(f'  inicio filtro: {len(df)}')
#   df = df[~df['amplicon'].duplicated(keep=False)]
#   print(f'  filtro amplicons duplicados: {len(df)}')
#   df = df.query('tamanho_amplicon > 300')
#   print(f'  filtro tamanho do amplicon: {len(df)}')

#   total = len(df)
#   if total == 0:
#     return df

#   minimo = 100
#   maximo = int(total * 0.1)

#   if maximo < minimo:
#     maximo = minimo

#   if maximo > total:
#     maximo = total

#   # Seleciona aleatoriamente entre mínimo e máximo de amostras
#   # n_amostras = max(minimo, min(maximo, total))
#   # return df.sample(n=n_amostras, random_state=42)

#   # seleciona na ordem recebida
#   return df[0:maximo]

arquivos_amplicon = busca_arquivos('AMP')

dados = []
totais = 0
for idx, arquivo in enumerate(arquivos_amplicon):
  df_amplicons = pd.read_csv(arquivo, delimiter = ";")
  id_arquivo = quebra_nome_arquivo(arquivo)
  origem = id_arquivo[0]
  print(f'> lido: {arquivo}')

  df_amplicons['entropy_amplicon'] = df_amplicons['amplicon'].apply(shannon_entropy)
  df_amplicons['entropy_fragment'] = df_amplicons['fragment'].apply(shannon_entropy)

  print(f"df_amplicons: {len(df_amplicons)}")

  # # filtra apenas os amplicons que contêm o fragmento - entre o inicio e o fim do str
  # df_amplicons = df_amplicons[
  #   df_amplicons.apply(lambda row: row['fragment'] in row['amplicon'], axis=1)
  # ]
  # print(f"df_amplicons - filtra apenas os amplicons que contêm o fragment: {len(df_amplicons)}")

  # filtra posicoes
  df_amplicons = df_amplicons[
    (df_amplicons['primer_left_index'] <= 200) &
    (df_amplicons['primer_right_index'] >= (df_amplicons['montante'].str.len() + df_amplicons['fragment'].str.len() - 2))
  ]
  print(f"df_amplicons - filtra posicoes: {len(df_amplicons)}")



  # exclui unidades STR duplicados
  df_amplicons = df_amplicons.sort_values(by=['unidade', 'copias', 'primer_right_distancia'], ascending=[True, False, True])
  # df_amplicons = df_amplicons.sort_values(by=['unidade', 'copias'], ascending=[True, False])
  df_amplicons = df_amplicons.drop_duplicates(subset=['unidade', 'copias'], keep='first').reset_index(drop=True)
  print(f"df_amplicons - exclui STR duplicados: {len(df_amplicons)}")

  # exclui amplicons duplicados
  df_amplicons = df_amplicons.sort_values(by=['amplicon', 'tamanho_str', 'primer_right_distancia', 'tamanho_amplicon'], ascending=[True, False, True, False])
  df_amplicons = df_amplicons.drop_duplicates(subset=['amplicon'], keep='first').reset_index(drop=True)
  print(f"df_amplicons - exclui amplicons duplicados: {len(df_amplicons)}")

  # ordem final
  # df_amplicons = df_amplicons.sort_values(by=['tamanho_str', 'tamanho_amplicon'], ascending=[False, False])
  df_amplicons = df_amplicons.sort_values(by=['tamanho_str'], ascending=[False])
  df_amplicons = df_amplicons.sort_values(by=['tamanho_str', 'score'], ascending=[False, False])

  # limita quantidade final de registros
  # df_filtrado = filtrar_amostras(df_amplicons) # COM filtro
  maximo = 100
  if len(df_amplicons) < 100:
    maximo = len(df_amplicons)
  df_filtrado = df_amplicons[0:maximo]
  # df_filtrado = df_amplicons # SEM filtro

  url_arquivo =  f'{pasta_analise}{origem}_FIM.csv'
  df_filtrado = df_filtrado.reset_index(drop=True)
  df_filtrado.insert(0, 'index_amp', range(1, len(df_filtrado) + 1))
  df_filtrado.to_csv(url_arquivo, sep=';', index=False)
  print(f'   > gravado/filtrados: {url_arquivo} ({len(df_filtrado)} de {len(df_amplicons)}) - {datetime.now()}')

  dados.append(df_filtrado)
  totais = totais + len(df_amplicons)

# df_geral = pd.concat(dados, ignore_index=False)
# url_arquivo_geral =  f'{pasta_analise}FIM_full.csv'
# df_geral = df_geral.reset_index()
# df_geral = df_geral.rename(columns={'index': 'fk_'})
# df_geral.to_csv(url_arquivo_geral, sep=';', index=False)
# print(f'>>> fim - full - gravado: {url_arquivo_geral} ({len(df_geral)} de {totais}) - {datetime.now()}')
print(f'>>> fim')

# > lido: /content/drive/MyDrive/pesquisa/2025-09-04/ASM1928827v1_AMP.csv
# df_amplicons: 5698
# df_amplicons - STR duplicados: 1066
# df_amplicons - exclui amplicons duplicados: 1047
#   inicio filtro: 1047
#   filtro amplicons duplicados: 1047
#   filtro tamanho do amplicon: 184
#    > gravado/filtrados: /content/drive/MyDrive/pesquisa/2025-09-04/ASM1928827v1_FIM.csv (100 de 1047) - 2025-11-07 16:52:23.302261
# > lido: /content/drive/MyDrive/pesquisa/2025-09-04/ASM285v2_AMP.csv
# df_amplicons: 5328
# df_amplicons - STR duplicados: 1029
# df_amplicons - exclui amplicons duplicados: 1021
#   inicio filtro: 1021
#   filtro amplicons duplicados: 1021
#   filtro tamanho do amplicon: 187
#    > gravado/filtrados: /content/drive/MyDrive/pesquisa/2025-09-04/ASM285v2_FIM.csv (100 de 1021) - 2025-11-07 16:52:23.591699
# > lido: /content/drive/MyDrive/pesquisa/2025-09-04/ASM2978390v1_AMP.csv
# df_amplicons: 4402
# df_amplicons - STR duplicados: 822
# df_amplicons - exclui amplicons duplicados: 815
#   inicio filtro: 815
#   filtro amplicons duplicados: 815
#   filtro tamanho do amplicon: 156
#    > gravado/filtrados: /content/drive/MyDrive/pesquisa/2025-09-04/ASM2978390v1_FIM.csv (100 de 815) - 2025-11-07 16:52:24.502141
# > lido: /content/drive/MyDrive/pesquisa/2025-09-04/ASM2978392v1_AMP.csv
# df_amplicons: 2259
# df_amplicons - STR duplicados: 410
# df_amplicons - exclui amplicons duplicados: 408
#   inicio filtro: 408
#   filtro amplicons duplicados: 408
#   filtro tamanho do amplicon: 84
#    > gravado/filtrados: /content/drive/MyDrive/pesquisa/2025-09-04/ASM2978392v1_FIM.csv (84 de 408) - 2025-11-07 16:52:24.637469
# > lido: /content/drive/MyDrive/pesquisa/2025-09-04/ASM4765177v1_AMP.csv
# df_amplicons: 3781
# df_amplicons - STR duplicados: 666
# df_amplicons - exclui amplicons duplicados: 655
#   inicio filtro: 655
#   filtro amplicons duplicados: 655
#   filtro tamanho do amplicon: 128
#    > gravado/filtrados: /content/drive/MyDrive/pesquisa/2025-09-04/ASM4765177v1_FIM.csv (100 de 655) - 2025-11-07 16:52:24.837377
# >>> fim

In [ ]:
# PASSO 11 - Gerar arquivo FASTA para conferir com BlastN NCBI

import os
import glob

# excluir todos os arquivos FASTA (*_FIM.fasta) da pasta de trabalho
for fasta_file in glob.glob(os.path.join(pasta_analise, "*_FIM.fasta")):
  os.remove(fasta_file)
  print(f"  > excluído {fasta_file}")

arquivos_amplicons = busca_arquivos('_FIM', ordenar_por="nome", reverso=True)
for idx_arquivo, arquivo in enumerate(arquivos_amplicons):
  df_amplicons = pd.read_csv(arquivo, delimiter = ";")
  print(f'  > lido: {arquivo} ({len(df_amplicons)})')

  for _, row in df_amplicons.iterrows():
    origem = row['origem']
    id_seq = f"{row['index_amp']}-{row['unidade']}-{row['copias']}"
    seq = row["amplicon"]
    url_fasta = os.path.join(pasta_analise, f"{origem}_FIM.fasta")
    with open(url_fasta, "a") as f:
          f.write(f">{id_seq}\n{seq}\n")
  print(f"  > gravado {url_fasta}")
print(f">>> fim")

In [ ]:
# busca rapida de arquivos
# *_FIM.fasta Alignment

import os
import glob
import datetime

print(f"data atual: {datetime.datetime.now()}")

# tipo_arquivo = "*_FIM.fasta"
tipo_arquivo = "*Alignment.xml"
# tipo_arquivo = "*_BLASTN*.csv"-
# tipo_arquivo = "*_PRIMER.csv*"
# tipo_arquivo = "*BLASTN.csv"

arquivos = glob.glob(os.path.join(pasta_analise, tipo_arquivo))
print(f"arquivos localizados: {len(arquivos)}")
for arquivo in arquivos:
  timestamp_criacao = os.path.getctime(arquivo)
  data_hora_criacao = datetime.datetime.fromtimestamp(timestamp_criacao)
  print(f'{arquivo} {data_hora_criacao}')
  # os.remove(arquivo)

In [ ]:
# mover arquivos

import os
import shutil
from glob import glob
from time import sleep
import datetime

# Caminho base onde os arquivos foram movidos
pasta_arquivos = os.path.join(pasta_analise, 'fim_blast')
# pasta_arquivos = os.path.join(pasta_analise, 'fim_blast')
# pasta_arquivos = os.path.join(pasta_analise, 'blast_result')
if not os.path.exists(pasta_arquivos):
    os.makedirs(pasta_arquivos)
    sleep(2)

# tipo_arquivo = "*_FIM.fasta"
# tipo_arquivo = "*Alignment.xml"
tipo_arquivo = "*BLASTN*.csv"
# tipo_arquivo = "*_PRIMER.csv*"

arquivos_localizados = glob(os.path.join(pasta_analise, tipo_arquivo))
print(f"arquivos_localizados: {len(arquivos_localizados)}")
for arquivo in arquivos_localizados:
  timestamp_criacao = os.path.getctime(arquivo)
  data_hora_criacao = datetime.datetime.fromtimestamp(timestamp_criacao)
  destino = os.path.join(pasta_arquivos, os.path.basename(arquivo))
  # shutil.move(arquivo, destino)
  # os.remove(arquivo)
  print(f'   Movido para: {arquivo} → {destino}')
  print(f'     Criação: {data_hora_criacao}')


In [16]:
# PASSO 12 - Ler arquivos (XML) retornados do BlastN NCBI e filtrar os alinhamentos
!pip install biopython
import os
from Bio.Blast import NCBIXML

# arquivos_blastn = busca_arquivos('ASM285v2_HRCSWB4N016-Alignment', extensao='xml', ordenar_por="nome", reverso=True)
arquivos_blastn = busca_arquivos('Alignment', extensao='xml', ordenar_por="nome", reverso=True)
print(f"{len(arquivos_blastn)} arquivos blastn localizados.")

for idx_arquivo, arquivo in enumerate(arquivos_blastn):
  # df_amplicons = pd.read_csv(arquivo, delimiter = ";")
  print(f'  > lido: {arquivo})')
  nome_arquivo = os.path.basename(arquivo) # Obter append o nome do arquivo sem o diretório
  origem = nome_arquivo.split('_')[0]
  dicionario = df_amostras.set_index("id")["Cepa (strain)"].to_dict()
  cepa = dicionario.get(origem)
  dicionario_loc = df_amostras.set_index("id")["Cepa (strain)"].to_dict()
  cepa_origem = dicionario_loc.get(origem)
  # print(f'  > lido: {arquivo} ({len(df_amplicons)})')

  # abre e faz o parsing
  with open(arquivo) as handle:
      blast_records = list(NCBIXML.parse(handle))  # lista com um registro por query

  print(f"   > {origem} Total de sequências consultadas (queries): {len(blast_records)}")

  # busca arquivo amplicons que gerou os arquivo .fasta
  arquivo_amplicons = f'{pasta_analise}{origem}_FIM.csv'
  df_amplicons = pd.read_csv(arquivo_amplicons, delimiter = ";")
  print(f"arquivos amplicons localizado: {arquivos_blastn}")

  # percorrer cada sequência da query
  resultados = []
  for index_record, record in enumerate(blast_records, start=1):
      query_id = record.query.split()[0]
      sequencial_id = int(query_id.split('-')[0])
      unidade = query_id.split('-')[1]
      copias = query_id.split('-')[2]
      amplicon = getattr(record, "query_sequence", "")

      df_query = df_amplicons.query("index_amp == @sequencial_id")
      if not df_query.empty:
        _df_amplicom = df_query.iloc[0]
      else:
        print(f"   {origem} ⚠️ Nenhuma linha encontrada com ID == {sequencial_id} no arquivo {arquivo_amplicons}")
        continue  # pula para a próxima sequência
      # mostrar o melhor alinhamento (se existir)
      if record.alignments:
          best_alignment = record.alignments[0]
          hsp = best_alignment.hsps[0]
          # print(f"      > Acesso (accession): {best_alignment.accession}")
          # print(f"      > Título: {best_alignment.title}")
          # print(f"      > Identidade: {hsp.identities}/{hsp.align_length} ({hsp.identities / hsp.align_length:.1%})")
          # print(f"      > E-value: {hsp.expect}")

          sequencial_alinhamento = 0
          quantidade_cepas_localizadas = 0
          ultima_cepa_localizada = {}
          for alignment in record.alignments:
            for hsp in alignment.hsps:
              sequencial_alinhamento = sequencial_alinhamento + 1

              cepa_loc = ""
              is_cepa_localizada = False
              if cepa_origem in alignment.title:
                is_cepa_localizada = True
                cepa_loc = cepa_origem
              else:
                if origem == 'ASM285v2':
                  print(f'{origem} - {alignment.accession}')
                if alignment.accession in df_amostras.query('id == @origem')["Scaffolds"]:
                  is_cepa_localizada = True
                  cepa_loc = alignment.accession

              # tamanho_amplicon = len(_df_amplicom['amplicon'])
              tamanho_amplicon = _df_amplicom['tamanho_amplicon']
              is_tamanhos_exatos = False
              # if hsp.align_length == hsp.identities:
              if hsp.align_length == tamanho_amplicon:
                is_tamanhos_exatos = True

              percent_identity = (hsp.identities / hsp.align_length) * 100

              # Um amplicon pode alinhar em mais de uma organismo/cepa do banco de dados
              #     - nome completo da cepa (alinhamento somente com a cepa verdadeira)
              #         is_cepa_localizada
              #     - tamanhos exatos entre o resultado de alinhamento x identities
              #         hsp.align_length == hsp.identities ou hsp.align_length == tamanho_amplicon
              #     - identidade
              #         percent_identity == 100
              is_identico = False
              if is_cepa_localizada and is_tamanhos_exatos and percent_identity == 100:
                is_identico = True

              novaLinha = _df_amplicom.to_dict()

              novaLinha['origem_'] = origem;
              novaLinha['cepa'] = cepa;
              novaLinha['texto_cepa_localizada'] = cepa_loc;
              novaLinha['unidade_'] = unidade;
              novaLinha['copias_s'] = copias;
              novaLinha['query_id'] = query_id;
              novaLinha['query_record'] = record.query;
              novaLinha['numero_alinhamentos'] = len(record.alignments);
              novaLinha['sequencial_alinhamento'] = sequencial_alinhamento;
              novaLinha['accession_number'] = alignment.accession;
              novaLinha['alignment_title'] = alignment.title;
              novaLinha['alignment_hit_id'] = alignment.hit_id;
              novaLinha['alignment_hit_def'] = alignment.hit_def;
              novaLinha['alignment_length'] = alignment.length;
              novaLinha['score'] = hsp.score;
              novaLinha['bits'] = hsp.bits;
              novaLinha['e-value'] = hsp.expect;
              novaLinha['query'] = hsp.query;
              novaLinha['match'] = hsp.match;
              novaLinha['sbjct'] = hsp.sbjct;
              novaLinha['query_start'] = hsp.query_start;
              novaLinha['query_end'] = hsp.query_end;
              novaLinha['sbjct_start'] = hsp.sbjct_start;
              novaLinha['sbjct_end'] = hsp.sbjct_end;
              novaLinha['identities'] = hsp.identities;
              novaLinha['positives'] = hsp.positives;
              novaLinha['gaps'] = hsp.gaps;
              novaLinha['align_length'] = hsp.align_length;
              novaLinha['percent_identity'] = percent_identity;
              novaLinha['is_cepa_localizada'] = is_cepa_localizada;
              novaLinha['is_tamanhos_exatos'] = is_tamanhos_exatos;
              novaLinha['is_identico'] = is_identico;

              resultados.append(novaLinha)
      else:
          print(f"      {origem} ⚠️ Nenhum alinhamento encontrado. origem: {origem}, unidade: {unidade}, copias: {copias}")

  df_alinhamentos = pd.DataFrame(resultados)
  url_arquivo = f'{pasta_analise}{origem}_BLASTN.csv'
  df_alinhamentos.to_csv(url_arquivo, sep=';', index=False)
  print(f'  > gravado: {url_arquivo} ({len(df_alinhamentos)})')

print(f'>>> fim')


5 arquivos blastn localizados.
  > lido: /content/drive/MyDrive/pesquisa/2025-09-04/ASM4765177v1_HBMEFTE6014-Alignment.xml)
NCBIXML: Ignored: '\nCREATE_VIEW\n\n\n'
NCBIXML: Ignored: '\nCREATE_VIEW\n\n\n'
NCBIXML: Ignored: '\nCREATE_VIEW\n\n\n'
NCBIXML: Ignored: '\nCREATE_VIEW\n\n\n'
   > ASM4765177v1 Total de sequências consultadas (queries): 100
arquivos amplicons localizado: ['/content/drive/MyDrive/pesquisa/2025-09-04/ASM4765177v1_HBMEFTE6014-Alignment.xml', '/content/drive/MyDrive/pesquisa/2025-09-04/ASM2978392v1_HBM6CA75014-Alignment.xml', '/content/drive/MyDrive/pesquisa/2025-09-04/ASM2978390v1_HBM3H5EF014-Alignment.xml', '/content/drive/MyDrive/pesquisa/2025-09-04/ASM285v2_refseqgenomes-M402K4SV014-Alignment.xml', '/content/drive/MyDrive/pesquisa/2025-09-04/ASM1928827v1_HBKJYJJ9014-Alignment.xml']
      ASM4765177v1 ⚠️ Nenhum alinhamento encontrado. origem: ASM4765177v1, unidade: AAACCCTA, copias: 12
      ASM4765177v1 ⚠️ Nenhum alinhamento encontrado. origem: ASM4765177v1, unid

In [14]:
# ASM285v2_scaffolds_validos = [
#     "NT_166518", "NT_166519", "NT_166520", "NT_166521",
#     "NT_166522", "NT_166523", "NT_166524", "NT_166525",
#     "NT_166526", "NT_166527", "NT_166528", "NT_166529",
#     "NT_166530", "NT_166531", "NT_166532", "NT_166533",
#     "NT_166537", "NT_166538", "NT_166539"
# ]
df_amostras
# df_amostras.query("id == 'ASM285v2'")["Scaffolds"]
# if alignment.accession in df_amostras.query('id == ASM285v2')["Scaffolds"]:
#   is_cepa_localizada = True
#   cepa_loc = alignment.hit_id

#   accession_number

,id,Nome da Montagem,Referência GenBank,WGS Accession,Cepa (strain),Scaffolds,Submissor,Nível de montagem,Tamanho (Mb),Dados
0,ASM285v2,ASM285v2 *,GCA_000002855.2,PRJNA19275,CBS 513.88,"[NT_166518, NT_166519, NT_166520, NT_166521, NT_166522, NT_166523, NT_166524, NT_166525, NT_166526, NT_166527, NT_166528, NT_166529, NT_166530, NT_166531, NT_166532, NT_166533, NT_166537, NT_166538, NT_166539]","DSM, The Netherlands",Scaffold,34,/content/drive/MyDrive/fasta/aspergillus/ASM285v2/ncbi_dataset/data/GCA_000002855.2/GCA_000002855.2_ASM285v2_genomic.fna
1,ASM1928827v1,ASM1928827v1,GCA_019288275.1,JAGRPH01,CBS 554.65,[],TU Wien,Chromosome,40,/content/drive/MyDrive/fasta/aspergillus/ASM1928827v1/ncbi_dataset/data/GCA_019288275.1/GCA_019288275.1_ASM1928827v1_genomic.fna
2,ASM2978390v1,ASM2978390v1,GCA_029783905.1,JAPVRD01,KJC3,[],Soongsil University,Chromosome,40,/content/drive/MyDrive/fasta/aspergillus/ASM2978390v1/ncbi_dataset/data/GCA_029783905.1/GCA_029783905.1_ASM2978390v1_genomic.fna
3,ASM2978392v1,ASM2978392v1,GCA_029783925.1,JAPVRE01,KYF3,[],Soongsil University,Chromosome,37,/content/drive/MyDrive/fasta/aspergillus/ASM2978392v1/ncbi_dataset/data/GCA_029783925.1/GCA_029783925.1_ASM2978392v1_genomic.fna
4,ASM4765177v1,ASM4765177v1,GCA_047651775.1,JBKZXA01,CCTCC 206047,[],Zhejiang Uni.Technology,Complete,35,/content/drive/MyDrive/fasta/aspergillus/ASM4765177v1/ncbi_dataset/data/GCA_047651775.1/GCA_047651775.1_ASM4765177v1_genomic.fna


In [11]:
# PASSO 14.1 filtro sobre os resultados

import os
import glob
import datetime

tipo_arquivo = "*BLASTN.csv"

arquivos = glob.glob(os.path.join(pasta_analise, tipo_arquivo))
print(f"arquivos localizados: {len(arquivos)}")

for arquivo in arquivos:
  timestamp_criacao = os.path.getctime(arquivo)
  data_hora_criacao = datetime.datetime.fromtimestamp(timestamp_criacao)
  print(f'{arquivo} {data_hora_criacao}')

  nome_arquivo = os.path.basename(arquivo)
  origem = nome_arquivo.split('_')[0]

  df = pd.read_csv(arquivo, delimiter = ";")
  # 1 - Filtrar pelos critérios
  df_filtrado = df[
      (df["percent_identity"] == 100)
      # & (df["align_length"] == df["tamanho_amplicon"])
      & (df["is_tamanhos_exatos"] == True)
  ]

  # url_arquivo_filtro_1 = f'{pasta_analise}{origem}_BLASTN-FILTRO-1.csv'
  # df_filtrado.to_csv(url_arquivo_filtro_1, sep=';', index=False)
  # print(f'  > gravado: {url_arquivo_filtro_1} ({len(df_filtrado)})')

  # 2 - remover duplicidade de cepa localizada
  # -------------------------------------------------------
  # 2) APLICAR A REGRA PRINCIPAL:
  #    "Se houver QUALQUER outra cepa, descarta o index_amp"
  # -------------------------------------------------------

  # index_amp válidos são aqueles onde TODO MUNDO é localizado
  index_validos = (
      df_filtrado
      .groupby("index_amp")["is_cepa_localizada"]
      .apply(lambda s: s.all())     # True só se TODAS as linhas forem True
      .loc[lambda s: s]             # mantém apenas True
      .index
  )
  # -------------------------------------------------------
  # 3) FILTRAR APENAS OS index_amp VÁLIDOS
  # -------------------------------------------------------
  df_apenas_localizada = df_filtrado[df_filtrado["index_amp"].isin(index_validos)]

  # -------------------------------------------------------
  # 4) REMOVER DUPLICATAS DENTRO DA MESMA CEPA LOCALIZADA (opcional)
  # -------------------------------------------------------
  df_resultado = df_apenas_localizada.drop_duplicates(
      subset=["index_amp"],  # só para garantir um registro por index_amp
      keep="first"
  )

  url_arquivo_filtro_2 = f'{pasta_analise}{origem}_BLASTN-FILTRO.csv'
  df_resultado.to_csv(url_arquivo_filtro_2, sep=';', index=False)
  print(f'  > gravado: {url_arquivo_filtro_2} ({len(df_resultado)})')
print(f'>> fim')

arquivos localizados: 5
/content/drive/MyDrive/pesquisa/2025-09-04/ASM4765177v1_BLASTN.csv 2025-12-18 16:35:50
  > gravado: /content/drive/MyDrive/pesquisa/2025-09-04/ASM4765177v1_BLASTN-FILTRO.csv (18)
/content/drive/MyDrive/pesquisa/2025-09-04/ASM2978392v1_BLASTN.csv 2025-12-18 16:44:24
  > gravado: /content/drive/MyDrive/pesquisa/2025-09-04/ASM2978392v1_BLASTN-FILTRO.csv (30)
/content/drive/MyDrive/pesquisa/2025-09-04/ASM2978390v1_BLASTN.csv 2025-12-18 16:44:53
  > gravado: /content/drive/MyDrive/pesquisa/2025-09-04/ASM2978390v1_BLASTN-FILTRO.csv (56)
/content/drive/MyDrive/pesquisa/2025-09-04/ASM285v2_BLASTN.csv 2025-12-18 17:04:31
  > gravado: /content/drive/MyDrive/pesquisa/2025-09-04/ASM285v2_BLASTN-FILTRO.csv (0)
/content/drive/MyDrive/pesquisa/2025-09-04/ASM1928827v1_BLASTN.csv 2025-12-18 17:04:31
  > gravado: /content/drive/MyDrive/pesquisa/2025-09-04/ASM1928827v1_BLASTN-FILTRO.csv (43)
>> fim


In [ ]:
# passo 13 - validar cepa principal pois o ban

scaffolds_validos = {
    "NT_166518.1", "NT_166519.1", "NT_166520.1", "NT_166521.1",
    "NT_166522.1", "NT_166523.1", "NT_166524.1", "NT_166525.1",
    "NT_166526.1", "NT_166527.1", "NT_166528.1", "NT_166529.1",
    "NT_166530.1", "NT_166531.1", "NT_166532.1", "NT_166533.1",
    "NT_166537.1", "NT_166538.4", "NT_166539.1"
}

def pertence_asm285(hit_id, hit_def):
    texto = f"{hit_id} {hit_def}"
    return any(acc in texto for acc in scaffolds_validos)

In [ ]:
# PASSO 14 tabelas de resultado - tabela 6

import pandas as pd
import os
import glob
import datetime

tipo_arquivo = "*_BLASTN-FILTRO.csv"
arquivos = glob.glob(os.path.join(pasta_analise, tipo_arquivo))
print(f"arquivos localizados: {len(arquivos)}")

for arquivo in arquivos:
  df = pd.read_csv(arquivo, sep=";")
  print(f'  > lido: {arquivo})')
  nome_arquivo = os.path.basename(arquivo)
  origem = nome_arquivo.split('_')[0]

  # criar IDs para os LOCUS (STR01, STR02, ...)
  df = df.reset_index(drop=True)
  df["Locus"] = [f"STR{i+1:02d}" for i in range(len(df))]
  # print(df)

  colunas_resumo = [
      "Locus",
      "unidade",
      "inicio_loci",
      "fim_loci",
      "copias",
      "tamanho_amplicon",
      "score",
      "e-value"
  ]

  tabela_principal = df[colunas_resumo].copy()

  # Renomear colunas para apresentação
  tabela_principal = tabela_principal.rename(columns={
      "unidade": "Unidade STR",
      "inicio_loci": "Início",
      "fim_loci": "Fim",
      "copias": "Nº de cópias",
      "tamanho_amplicon": "Tamanho (bp)",
      "score": "Score",
      "e-value": "E-value"
  })

  colunas_suplementar = [
      "Locus",
      "primer_left",
      "primer_right",
      "amplicon"
  ]

  tabela_suplementar = df[colunas_suplementar].copy()

  tabela_suplementar = tabela_suplementar.rename(columns={
      "primer_left": "Primer esquerdo (5'–3')",
      "primer_right": "Primer direito (5'–3')",
      "amplicon": "Amplicon (5'–3')"
  })

  url_arquivo_resumo = f'{pasta_analise}{origem}_TABELA-RESUMO.csv'
  tabela_principal.to_csv(url_arquivo_resumo, sep=';', index=False)
  print(f'  > gravado: {url_arquivo_resumo} ({len(tabela_principal)})')

  url_arquivo_suplementar = f'{pasta_analise}{origem}_TABELA-SUPLEMENTAR.csv'
  tabela_suplementar.to_csv(url_arquivo_suplementar, sep=';', index=False)
  print(f'  > gravado: {url_arquivo_suplementar} ({len(tabela_suplementar)})')

print(">>> fim")

In [ ]:
print(df.columns.tolist())

In [ ]:
# analise sobre a saída
import os
import pandas as pd

print("Total de alinhamentos positivos")
arquivos_blastn = busca_arquivos('_BLASTN_FILTRO', extensao='csv', ordenar_por="nome", reverso=True)
for idx_arquivo, arquivo in enumerate(arquivos_blastn):
  if os.path.getsize(arquivo) == 0:
    print(f"   > ignorado (vazio): {nome_arquivo}")
    continue

  nome_arquivo = os.path.basename(arquivo) # Obter apenas o nome do arquivo sem o diretório
  origem = nome_arquivo.split('_')[0]

  try:
    df_alinhamentos = pd.read_csv(arquivo, delimiter = ";")
    print(f"   > lido: {nome_arquivo}")
    if 'index_amp' in df_alinhamentos.columns:
        total_alinhamentos = df_alinhamentos['index_amp'].nunique()
        print(f"   > {origem}: {total_alinhamentos} de {len(df_alinhamentos)}")
    else:
        print(f"   > {origem}: coluna 'index_amp' não encontrada")
  except pd.errors.EmptyDataError:
    print(f"   > erro ao ler (sem dados): {nome_arquivo}")

In [ ]:
# PASSO 13 - gerar imagens -> visualizar STR, flankeadas primers e amplicon
# 05-11-2025

def safe_str(val):
    return "" if pd.isna(val) else str(val)

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.patches import Patch

import pandas as pd
import os
# from str_utils import busca_arquivos, busca_sequencia_fasta, create_flank, gravar_arquivo_saida

arquivos_blastn = busca_arquivos('_FIM', ordenar_por="nome", reverso=True)
for index_arquivo, arquivo in enumerate(arquivos_blastn):
  # df_amplicons = pd.read_csv(arquivo, delimiter = ";")
  print(f'  > lido: {arquivo}')
  nome_arquivo = os.path.basename(arquivo) # Obter apenas o nome do arquivo sem o diretório
  origem = nome_arquivo.split('_')[0]
  df_blast = pd.read_csv(arquivo, delimiter = ";")

  for index_blast, alinhamento in df_blast.iterrows():
    borda_esquerda = safe_str(alinhamento['borda_esquerda'])
    montante       = safe_str(alinhamento['montante'])
    str_seq        = safe_str(alinhamento['fragment'])
    jusante        = safe_str(alinhamento['jusante'])
    borda_right    = safe_str(alinhamento['borda_direita'])

# ajuste acima
    # borda_esquerda = alinhamento['borda_esquerda']
    # montante = alinhamento['montante']
    # str_seq = alinhamento['fragment']
    # jusante = alinhamento['jusante']
    # # borda_right = alinhamento['borda_direita'] aki
    # borda_right = "" if pd.isna(borda_right) else str(borda_right)

    unidade_str = alinhamento['unidade']
    primer_left = alinhamento['primer_left']
    primer_right = alinhamento['primer_right']
    primer_forward = alinhamento['primer_right']
    tipo_str = alinhamento['tipo']
    offset = alinhamento['inicio_amostral']
    amplicon = alinhamento['amplicon']
    # origem = alinhamento['origem']
    index_amp = alinhamento['index_amp']

    pasta_imagens = os.path.join(pasta_analise, 'imagens', origem)
    print(f'pasta_imagens: {pasta_imagens}')
    arquivo_png = os.path.join(pasta_imagens, f"{origem}_schema_{index_amp}.png")
    if os.path.exists(arquivo_png):
      print(f"Arquivo já existe: {arquivo_png}")
      pasta_imagens
      continue
    if not os.path.exists(pasta_imagens):
        os.makedirs(pasta_imagens)
        sleep(10)

    #  > fragment:	Sequência entre inicio e fim
    #  > montante:	Região anterior ao fragmento, com tamanho mínimo de 200
    #  > jusante:	Região posterior ao fragmento, com tamanho mínimo de 200
    #  > full_flank:	Flanqueadoras + fragmento central
    #  > borda_esquerda:	Região antes da flanqueadora esquerda (200 bases)
    #  > borda_direita:	Região após a flanqueadora direita (200 bases)
    #  > inicio_amostral:	Índice inicial da sequência total estendida
    #  > fim_amostral:	Índice final da sequência total estendida

    try:
      seq = str(borda_esquerda) + str(montante) + str(str_seq) + str(jusante) + str(borda_right)
      seq = seq.upper()
    except Exception as e:
      print("Erro ao concatenar seq:", e)
      print(f"borda_esquerda: {borda_esquerda} | montante: {montante} | str_seq: {str_seq} | jusante: {jusante} | borda_right: {borda_right}")
      seq = ""  # fallback vazio


    # print(f'seq: {len(seq)} borda_esquerda: {len(borda_esquerda)} montante: {len(montante)} str_seq: {len(str_seq)} jusante: {len(jusante)} borda_right: {len(borda_right)}')

    # Posições das regiões
    montante_start = len(borda_esquerda)
    montante_end = montante_start + len(montante)
    str_start = montante_end
    str_end = str_start + len(str_seq)
    jusante_start = str_end
    jusante_end = jusante_start + len(jusante)

    # print(f'montante_start: {montante_start} montante_end: {montante_end} str_start: {str_start} str_end: {str_end} jusante_start: {jusante_start} jusante_end: {jusante_end}')
    # print(f'amplicon ({len(amplicon)}): {amplicon}')
    # print(f'primer_left ({len(primer_left)}): {primer_left}')
    # print(f'primer_right ({len(primer_right)}): {primer_right}')
    # print(f"primer_right_loc ({len(alinhamento['primer_right_loc'])}): {alinhamento['primer_right_loc']}")

    # Posições dos primers dentro das regiões flanqueadoras
    primer_left_start = len(borda_esquerda) + alinhamento['primer_left_start']
    amplicon_start = primer_left_start
    amplicon_end = amplicon_start + len(amplicon)
    primer_left_end = primer_left_start + len(alinhamento['primer_left'])

    primer_forward_start = amplicon_end - len(primer_right)
    primer_forward_end = amplicon_end

    # Mapear bases para números
    bases = {'A':1, 'T':2, 'G':3, 'C':4, 'N':5}
    base_colors = {'A':'red', 'T':'blue', 'G':'green', 'C':'orange', 'N':'white'}

    # Criar stem plot
    fig, ax = plt.subplots(figsize=(25,4.5))

    # Fundo das regiões
    ax.axvspan(0, montante_start, color='lightgray', alpha=0.2)
    ax.axvspan(montante_start, montante_end, color='skyblue', alpha=0.2)
    ax.axvspan(str_start, str_end, color='gold', alpha=0.2)
    ax.axvspan(jusante_start, jusante_end, color='lightgreen', alpha=0.2)
    ax.axvspan(jusante_end, len(seq), color='lightgray', alpha=0.2)

    # linhas verticais pontilhadas
    # for pos in [montante_start, montante_end, str_end, jusante_end, primer_left_start, primer_left_end, primer_forward_start, primer_forward_end]:
    for pos in [montante_start, montante_end, jusante_start, jusante_end]:
        ax.axvline(x=pos, color='black', linestyle='--', linewidth=1)

    # Stem plot por base
    # for i, base in enumerate(seq):
    #     ax.stem([i+1], [bases[base]], linefmt=base_colors[base], markerfmt='', basefmt=" ")

    for i, base in enumerate(seq):
      b = base.upper()
      ax.stem([i+1],
              [bases.get(b, 0)],  # se não existir, usa 0
              linefmt=base_colors.get(b, 'gray'),  # se não existir, usa cor cinza
              markerfmt='',
              basefmt=" ")

    # Labels das regiões no topo
    ax.text(montante_start + len(montante)/2, 5.6, f"Montante ({len(montante)})", ha='center', fontsize=12, fontweight='bold')
    ax.text(str_start + len(str_seq)/2, 5.6, f'STR-{tipo_str}: {unidade_str}  ({len(str_seq)})', ha='center', fontsize=12, fontweight='bold')
    ax.text(jusante_start + len(jusante)/2, 5.6, f"Jusante ({len(jusante)})", ha='center', fontsize=12, fontweight='bold')

    # Labels dos primers acima dos retângulos dos primers
    ax.text(primer_left_start + (primer_left_end-primer_left_start)/2, 5.1, f"Primer Left ({len(primer_left)})", ha='center', fontsize=12, fontweight='bold', color='purple')
    ax.text(primer_forward_start + (primer_forward_end-primer_forward_start)/2, 5.1, f"Primer Right ({len(primer_forward)})", ha='center', fontsize=12, fontweight='bold', color='brown')

    # Primer Left e Forward abaixo de regiões flankeadoras
    primer_y_bottom = 4.7
    primer_y_top = 5
    ax.fill_betweenx([primer_y_bottom, primer_y_top], primer_left_start, primer_left_end, color='purple', alpha=0.6)
    ax.fill_betweenx([primer_y_bottom, primer_y_top], primer_forward_start, primer_forward_end, color='brown', alpha=0.6)

    # Legenda do primer_left
    hd_l = hamming_distance(alinhamento['primer_left'], alinhamento['primer_left_loc'])
    ax.text(primer_left_start - 3, primer_y_top -0.2, f"{alinhamento['primer_left']}",
            ha='right', va='center', fontname='monospace', fontsize=12, color='purple')
    label_primer_left_loc = f"{alinhamento['primer_left_loc']})"
    if hd_l == 0:
      cor = 'purple'
    else:
      cor = 'red'
      label_primer_left_loc += f" ({hd_l})"

    ax.text(primer_left_start - 3, primer_y_top - 0.5, label_primer_left_loc,
            ha='right', va='center', fontname='monospace', fontsize=12, color=f'{cor}')

    # Legenda do primer_right
    hd_r = hamming_distance(alinhamento['primer_right'], alinhamento['primer_right_loc'])
    ax.text(primer_forward_end + 3, primer_y_top -0.2, f"{alinhamento['primer_right']}",
            ha='left', va='center', fontname='monospace', fontsize=12, color='brown')
    label_primer_right_loc = f"{alinhamento['primer_right_loc']}"
    if hd_r == 0:
      cor = 'brown'
    else:
      cor = 'red'
      label_primer_right_loc += f" ({hd_r})"
    ax.text(primer_forward_end + 3, primer_y_top -0.5, label_primer_right_loc, # primer_right_loc
            ha='left', va='center', fontname='monospace', fontsize=12, color=f'{cor}')


    # Amplicon (produto amplificado) abaixo dos primers
    amplicon_y_bottom = 4.1
    amplicon_y_top = 4.4 # aki linha abaixo
    ax.fill_betweenx([amplicon_y_bottom, amplicon_y_top], primer_left_start, amplicon_end, color='black', alpha=0.5)
    ax.text(primer_left_start + (primer_forward_end-primer_left_start)/2, 4.55, f"Amplicon ({len(amplicon)})", ha='center', fontsize=12, fontweight='bold', color='black')

    # Ajustes visuais
    ax.set_ylim(0,6)
    ax.set_yticks([1,2,3,4])
    ax.set_yticklabels(['A','T','G','C'])

    # ticks horizontais - x_ticks
    xticks_personalizados = [0,
                            montante_start,
                            primer_left_start,
                            primer_left_end,
                            montante_end,
                            str_start,
                            str_end,
                            primer_forward_start,
                            primer_forward_end,
                            jusante_end,
                            jusante_end + len(borda_right)]

    ax.set_xticks(xticks_personalizados)
    ax.set_xticklabels([''] * len(xticks_personalizados))  # oculta os rótulos padrão

    # Ajusta o offset na da posicao na sequencia original (x_tics)
    for i, tick in enumerate(xticks_personalizados):
        label = f"{int(tick + offset):,}".replace(",", ".")
        y_offset = -0.04
        if i in [2, 7]:
          y_offset = -0.08
        if i in [3, 8]:
          y_offset = -0.12
        ax.text(tick, y_offset, label,
                ha='center', va='top',
                transform=ax.get_xaxis_transform(),
                fontsize=10, color='black')

    ax.set_xlabel(f"Posição das porções destacadas na sequência FASTA original da amostra {origem}", labelpad=33, fontsize=12, fontweight='bold')
    ax.tick_params(axis='x', labelsize=14, width=2, length=8)

    ax.set_ylabel(f"Nucleotídeos", labelpad=5, fontsize=12, fontweight='bold')
    ax.tick_params(axis='y', labelsize=14, width=2, length=8)

    base_colors = {'A': 'red', 'C': 'orange', 'T': 'blue', 'G': 'green'}
    legend_bases = [Patch(color=base_colors[base], label=base) for base in ['A', 'T', 'G', 'C']]
    # legend_total = legend_bases + legend_areas
    ax.legend(handles=legend_bases , title="Bases", loc='upper left')

    plt.savefig(arquivo_png, bbox_inches='tight', pad_inches=0, dpi=300)
    print(f"   gravado: {arquivo_png}")
  # plt.show()
print(f">>> fim")

In [ ]:
# PASSO XX - Analise dos primers
# Montar aqruivo FASTA para testar primers contra uma sequência alvo para verificar especificidade e eficiência.

# https://www.ncbi.nlm.nih.gov/tools/primer-blast/





In [ ]:
past# PASSO 10.1 - visualizar STR, flankeadas primers e amplicon
# 15-09-2025 21-09-2025 10-10-2025 NameError: name 'primer_right' is not defined

import matplotlib.pyplot as plt
import random
import matplotlib.ticker as ticker
from matplotlib.patches import Patch

import pandas as pd
# from str_utils import busca_arquivos, busca_sequencia_fasta, create_flank, gravar_arquivo_saida

#####################################################
url_arquivo = pasta_analise + 'FIM_full.csv'
df_primer = pd.read_csv(url_arquivo, delimiter = ";")
df_primer = df_primer.sort_values(by='tamanho_amplicon', ascending=False)
primer_sel = df_primer.iloc[0]
#####################################################

borda_esquerda = primer_sel['borda_esquerda']
montante = primer_sel['montante']
str_seq = primer_sel['fragment']
jusante = primer_sel['jusante']
borda_right = primer_sel['borda_direita']
unidade_str = primer_sel['unidade']
primer_left = primer_sel['primer_left']
primer_right = primer_sel['primer_right']
primer_forward = primer_sel['primer_right']
tipo_str = primer_sel['tipo']
offset = primer_sel['inicio_amostral']
amplicon = primer_sel['amplicon']
origem = primer_sel['origem']

#  > fragment:	Sequência entre inicio e fim
#  > montante:	Região anterior ao fragmento, com tamanho mínimo de 200
#  > jusante:	Região posterior ao fragmento, com tamanho mínimo de 200
#  > full_flank:	Flanqueadoras + fragmento central
#  > borda_esquerda:	Região antes da flanqueadora esquerda (200 bases)
#  > borda_direita:	Região após a flanqueadora direita (200 bases)
#  > inicio_amostral:	Índice inicial da sequência total estendida
#  > fim_amostral:	Índice final da sequência total estendida

seq = borda_esquerda + montante + str_seq + jusante + borda_right
print(f'seq: {len(seq)} borda_esquerda: {len(borda_esquerda)} montante: {len(montante)} str_seq: {len(str_seq)} jusante: {len(jusante)} borda_right: {len(borda_right)}')

# Posições das regiões
montante_start = len(borda_esquerda)
montante_end = montante_start + len(montante)
str_start = montante_end
str_end = str_start + len(str_seq)
jusante_start = str_end
jusante_end = jusante_start + len(jusante)

print(f'montante_start: {montante_start} montante_end: {montante_end} str_start: {str_start} str_end: {str_end} jusante_start: {jusante_start} jusante_end: {jusante_end}')
print(f'amplicon ({len(amplicon)}): {amplicon}')
print(f'primer_left ({len(primer_left)}): {primer_left}')
print(f'primer_right ({len(primer_right)}): {primer_right}')
print(f"primer_right_loc ({len(primer_sel['primer_right_loc'])}): {primer_sel['primer_right_loc']}")

# Posições dos primers dentro das regiões flanqueadoras
primer_left_start = len(borda_esquerda) + primer_sel['primer_left_start']
amplicon_start = primer_left_start
amplicon_end = amplicon_start + len(amplicon)
primer_left_end = primer_left_start + len(primer_sel['primer_left'])

primer_forward_start = amplicon_end - len(primer_right)
primer_forward_end = amplicon_end


# Mapear bases para números
bases = {'A':1, 'T':2, 'G':3, 'C':4}
base_colors = {'A':'red', 'T':'blue', 'G':'green', 'C':'orange'}

# Criar stem plot
fig, ax = plt.subplots(figsize=(25,4.5))

# Fundo das regiões
ax.axvspan(0, montante_start, color='lightgray', alpha=0.2)
ax.axvspan(montante_start, montante_end, color='skyblue', alpha=0.2)
ax.axvspan(str_start, str_end, color='gold', alpha=0.2)
ax.axvspan(jusante_start, jusante_end, color='lightgreen', alpha=0.2)
ax.axvspan(jusante_end, len(seq), color='lightgray', alpha=0.2)

# linhas verticais pontilhadas
# for pos in [montante_start, montante_end, str_end, jusante_end, primer_left_start, primer_left_end, primer_forward_start, primer_forward_end]:
for pos in [montante_start, montante_end, jusante_start, jusante_end]:
    ax.axvline(x=pos, color='black', linestyle='--', linewidth=1)

# Stem plot por base
for i, base in enumerate(seq):
    ax.stem([i+1], [bases[base]], linefmt=base_colors[base], markerfmt='', basefmt=" ")

# Labels das regiões no topo
ax.text(montante_start + len(montante)/2, 5.6, f"Montante ({len(montante)})", ha='center', fontsize=12, fontweight='bold')
ax.text(str_start + len(str_seq)/2, 5.6, f'STR-{tipo_str}: {unidade_str}  ({len(str_seq)})', ha='center', fontsize=12, fontweight='bold')
ax.text(jusante_start + len(jusante)/2, 5.6, f"Jusante ({len(jusante)})", ha='center', fontsize=12, fontweight='bold')

# Labels dos primers acima dos retângulos dos primers
ax.text(primer_left_start + (primer_left_end-primer_left_start)/2, 5.1, f"Primer Left ({len(primer_left)})", ha='center', fontsize=12, fontweight='bold', color='purple')
ax.text(primer_forward_start + (primer_forward_end-primer_forward_start)/2, 5.1, f"Primer Right ({len(primer_forward)})", ha='center', fontsize=12, fontweight='bold', color='brown')

# Primer Left e Forward abaixo de regiões flankeadoras
primer_y_bottom = 4.7
primer_y_top = 5
ax.fill_betweenx([primer_y_bottom, primer_y_top], primer_left_start, primer_left_end, color='purple', alpha=0.6)
ax.fill_betweenx([primer_y_bottom, primer_y_top], primer_forward_start, primer_forward_end, color='brown', alpha=0.6)

# Legenda do primer_left
ax.text(primer_left_start - 3, primer_y_top -0.2, f"{primer_sel['primer_left']}",
        ha='right', va='center', fontname='monospace', fontsize=12, color='purple')
if primer_sel['primer_left'] == primer_sel['primer_left_loc']:
  cor = 'purple'
else:
  cor = 'red'
ax.text(primer_left_start - 3, primer_y_top - 0.5, f"{primer_sel['primer_left_loc']}",
        ha='right', va='center', fontname='monospace', fontsize=12, color=f'{cor}')

# Legenda do primer_right
ax.text(primer_forward_end + 3, primer_y_top -0.2, f"{primer_sel['primer_right']}",
        ha='left', va='center', fontname='monospace', fontsize=12, color='brown')
if primer_sel['primer_right'] == primer_sel['primer_right_loc']:
  cor = 'brown'
else:
  cor = 'red'
ax.text(primer_forward_end + 3, primer_y_top -0.5, f"{primer_sel['primer_right_loc']}", # primer_right_loc
        ha='left', va='center', fontname='monospace', fontsize=12, color=f'{cor}')


# Amplicon (produto amplificado) abaixo dos primers
amplicon_y_bottom = 4.1
amplicon_y_top = 4.4 # aki linha abaixo
ax.fill_betweenx([amplicon_y_bottom, amplicon_y_top], primer_left_start, amplicon_end, color='black', alpha=0.5)
ax.text(primer_left_start + (primer_forward_end-primer_left_start)/2, 4.55, f"Amplicon ({len(amplicon)})", ha='center', fontsize=12, fontweight='bold', color='black')

# Ajustes visuais
ax.set_ylim(0,6)
ax.set_yticks([1,2,3,4])
ax.set_yticklabels(['A','T','G','C'])

# ticks horizontais - x_ticks
xticks_personalizados = [0,
                         montante_start,
                         primer_left_start,
                         primer_left_end,
                         montante_end,
                         str_start,
                         str_end,
                         primer_forward_start,
                         primer_forward_end,
                         jusante_end,
                         jusante_end + len(borda_right)]

ax.set_xticks(xticks_personalizados)
ax.set_xticklabels([''] * len(xticks_personalizados))  # oculta os rótulos padrão

# Ajusta o offset na da posicao na sequencia original (x_tics)
for i, tick in enumerate(xticks_personalizados):
    label = f"{int(tick + offset):,}".replace(",", ".")
    y_offset = -0.04
    if i in [2, 7]:
      y_offset = -0.08
    if i in [3, 8]:
      y_offset = -0.12
    ax.text(tick, y_offset, label,
            ha='center', va='top',
            transform=ax.get_xaxis_transform(),
            fontsize=10, color='black')

ax.set_xlabel(f"Posição das porções destacadas na sequência FASTA original da amostra {origem}", labelpad=33, fontsize=12, fontweight='bold')
ax.tick_params(axis='x', labelsize=14, width=2, length=8)

ax.set_ylabel(f"Nucleotídeos", labelpad=5, fontsize=12, fontweight='bold')
ax.tick_params(axis='y', labelsize=14, width=2, length=8)

base_colors = {'A': 'red', 'C': 'orange', 'T': 'blue', 'G': 'green'}
legend_bases = [Patch(color=base_colors[base], label=base) for base in ['A', 'T', 'G', 'C']]
# legend_total = legend_bases + legend_areas
ax.legend(handles=legend_bases , title="Bases", loc='upper left')

arquivo_png = "/content/drive/MyDrive/pesquisa/desenho_primers_2.png"
plt.savefig(arquivo_png, bbox_inches='tight', pad_inches=0, dpi=300)

plt.show()

In [ ]:
# PASSO 10.1.a - visualizar primers - primer left em relaçao montante
# 15-09-2025 21-09-2025 25-09-2025

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import pandas as pd

#####################################################
url_arquivo = pasta_analise + 'FIM.csv'
df_primer = pd.read_csv(url_arquivo, delimiter = ";")
primer_sel = df_primer.iloc[0]
#####################################################
montante = primer_sel['montante']
offset = primer_sel['inicio_amostral'] + len(primer_sel['borda_esquerda'] )

#  > fragment:	Sequência entre inicio e fim
#  > montante:	Região anterior ao fragmento, com tamanho mínimo de 200
#  > jusante:	Região posterior ao fragmento, com tamanho mínimo de 200
#  > full_flank:	Flanqueadoras + fragmento central
#  > border_left:	Região antes da flanqueadora esquerda (200 bases)
#  > border_right:	Região após a flanqueadora direita (200 bases)
#  > inicio_amostral:	Índice inicial da sequência total estendida
#  > fim_amostral:	Índice final da sequência total estendida

seq = montante

# Posições das regiões
montante_start = 0
montante_end = montante_start + len(montante)

primer_left_start = primer_sel['primer_left_start']
primer_left_end = primer_left_start + len(primer_sel['primer_left'])

# Mapear bases para números
bases = {'A':1, 'T':2, 'G':3, 'C':4}
base_colors = {'A':'red', 'T':'blue', 'G':'green', 'C':'orange'}

# Criar stem plot
fig, ax = plt.subplots(figsize=(25,4.5))

# Fundo das regiões
ax.axvspan(primer_left_start + 0.5, primer_left_end + 0.5, color='purple', alpha=0.2)

# linhas verticais pontilhadas
for pos in [primer_left_start + 0.5, primer_left_end + 0.5]:
    ax.axvline(x=pos, color='black', linestyle='--', linewidth=1)

# Stem plot por base
for i, base in enumerate(seq):
    # ax.stem([i+1], [bases[base]], linefmt=base_colors[base], markerfmt='', basefmt=" "
    markerline, stemlines, baseline = ax.stem([i+1], [bases[base]], linefmt=base_colors[base], markerfmt='', basefmt=" ")
    plt.setp(stemlines, linewidth=5)

# Labels das regiões no topo
ax.text(montante_start + len(montante)/2, 5.6, f"Montante ({len(montante)})", ha='center', fontsize=12, fontweight='bold')

# Primer Left
primer_y_bottom = 4.7
primer_y_top = 5
ax.fill_betweenx([primer_y_bottom, primer_y_top], primer_left_start + 0.5, primer_left_end + 0.5, color='purple', alpha=0.6)

# Labels dos primers acima dos retângulos dos primers
ax.text(primer_left_start + (primer_left_end-primer_left_start)/2, 5.1, f"Primer Left {len(primer_sel['primer_left'])}", ha='center', fontsize=12, fontweight='bold', color='purple')

ax.text(primer_left_start - 3, primer_y_top -0.2, f"{primer_sel['primer_left']}",
        ha='right', va='center', fontname='monospace', fontsize=12, color='purple')
if primer_sel['primer_left'] == primer_sel['primer_left_loc']:
  cor = 'purple'
else:
  cor = 'red'
ax.text(primer_left_start - 3, primer_y_top - 0.5, f"{primer_sel['primer_left_loc']}",
        ha='right', va='center', fontname='monospace', fontsize=12, color=f'{cor}')

# Ajustes visuais
ax.set_ylim(0,6)
ax.set_yticks([1,2,3,4])
ax.set_yticklabels(['A','T','G','C'])


xticks_personalizados = [0,
                         montante_start,
                         primer_left_start,
                         primer_left_end,
                         montante_end]

ax.set_xticks(xticks_personalizados)
ax.set_xticklabels([''] * len(xticks_personalizados))  # oculta os rótulos padrão

# Ajusta o offset na da posicao na sequencia original (x_tics)
for i, tick in enumerate(xticks_personalizados):
    label = f"{int(tick + offset):,}".replace(",", ".")
    y_offset = -0.04
    ax.text(tick, y_offset, label,
            ha='center', va='top',
            transform=ax.get_xaxis_transform(),
            fontsize=10, color='black')

ax.set_xlabel(f"Posição das porções destacadas na sequência FASTA original da amostra {origem}", labelpad=33, fontsize=12, fontweight='bold')
ax.tick_params(axis='x', labelsize=14, width=2, length=8)

ax.set_ylabel(f"Nucleotídeos", labelpad=5, fontsize=12, fontweight='bold')
ax.tick_params(axis='y', labelsize=14, width=2, length=8)

base_colors = {'A': 'red', 'C': 'orange', 'T': 'blue', 'G': 'green'}
legend_bases = [Patch(color=base_colors[base], label=base) for base in ['A', 'T', 'G', 'C']]
# legend_total = legend_bases + legend_areas
ax.legend(handles=legend_bases , title="Bases", loc='upper left')


arquivo_png = "/content/drive/MyDrive/pesquisa/desenho_primer_left_montante.png"
plt.savefig(arquivo_png, bbox_inches='tight', pad_inches=0, dpi=300)

plt.show()

In [ ]:
# PASSO 10.1.b - visualizar primers - primer right em relaçao ao jusante
# 15-09-2025 21-09-2025 25-09-2025  ax.stem([i+1], [bases[base]], linefmt=base_colors[base], marke

import matplotlib.pyplot as plt
import random
import matplotlib.ticker as ticker

import pandas as pd

#####################################################
url_arquivo = pasta_analise + 'FIM.csv'
df_primer = pd.read_csv(url_arquivo, delimiter = ";")
primer_sel = df_primer.iloc[0]
#####################################################
montante = primer_sel['montante']
jusante = primer_sel['jusante']
str_seq = primer_sel['fragment']
offset = primer_sel['inicio_amostral'] + len(primer_sel['borda_esquerda']) + len(montante) + len(str_seq)

seq = jusante

# Posições das regiões
jusante_start = 0
jusante_end = jusante_start + len(jusante)

primer_right_start = primer_sel['primer_right_start'] - len(montante) - len(str_seq)
primer_right_end = primer_right_start + len(primer_sel['primer_right'])

# Mapear bases para números
bases = {'A':1, 'T':2, 'G':3, 'C':4}
base_colors = {'A':'red', 'T':'blue', 'G':'green', 'C':'orange'}

# Criar stem plot
fig, ax = plt.subplots(figsize=(25,4.5))

# Fundo das regiões
ax.axvspan(primer_right_start + 0.5, primer_right_end + 0.5, color='brown', alpha=0.2)

# linhas verticais pontilhadas
for pos in [primer_right_start + 0.5, primer_right_end + 0.5]:
    ax.axvline(x=pos, color='black', linestyle='--', linewidth=1)

# Stem plot por base/
for i, base in enumerate(seq):
    # ax.stem([i+1], [bases[base]], linefmt=base_colors[base], markerfmt='', basefmt=" ")
    markerline, stemlines, baseline = ax.stem([i+1], [bases[base]], linefmt=base_colors[base], markerfmt='', basefmt=" ")
    plt.setp(stemlines, linewidth=5)


# Labels das regiões no topo
ax.text(jusante_start + len(jusante)/2, 5.6, f"Jusante ({len(jusante)})", ha='center', fontsize=12, fontweight='bold')

# Labels dos primers acima dos retângulos dos primers
ax.text(primer_right_start + (primer_right_end-primer_right_start)/2, 5.1, f"Primer Right ({len(primer_forward)})", ha='center', fontsize=12, fontweight='bold', color='brown')

# Primer Right
primer_y_bottom = 4.7
primer_y_top = 5
ax.fill_betweenx([primer_y_bottom, primer_y_top], primer_right_start + 0.5, primer_right_end + 0.5, color='brown', alpha=0.6)

# Legenda do primer_forward (após o bloco)
ax.text(primer_right_end + 3, primer_y_top -0.2, f"{primer_sel['primer_right']}",
        ha='left', va='center', fontname='monospace', fontsize=12, color='brown')
if primer_sel['primer_right'] == primer_sel['primer_right_loc']:
  cor = 'brown'
else:
  cor = 'red'
ax.text(primer_right_end + 3, primer_y_top -0.5, f"{primer_sel['primer_right_loc']}", # primer_right_loc_direta
        ha='left', va='center', fontname='monospace', fontsize=12, color=f'{cor}')

# Ajustes visuais
ax.set_ylim(0,6)
ax.set_yticks([1,2,3,4])
ax.set_yticklabels(['A','T','G','C'])

xticks_personalizados = [0,
                         jusante_start,
                         primer_right_start,
                         primer_right_end,
                         jusante_end]

ax.set_xticks(xticks_personalizados)
ax.set_xticklabels([''] * len(xticks_personalizados))  # oculta os rótulos padrão

# Ajusta o offset na da posicao na sequencia original (x_tics)
for i, tick in enumerate(xticks_personalizados):
    label = f"{int(tick + offset):,}".replace(",", ".")
    y_offset = -0.04
    ax.text(tick, y_offset, label,
            ha='center', va='top',
            transform=ax.get_xaxis_transform(),
            fontsize=10, color='black')

ax.set_xlabel(f"Posição das porções destacadas na sequência FASTA original da amostra {origem}", labelpad=33, fontsize=12, fontweight='bold')
ax.tick_params(axis='x', labelsize=14, width=2, length=8)

ax.set_ylabel(f"Nucleotídeos", labelpad=5, fontsize=12, fontweight='bold')
ax.tick_params(axis='y', labelsize=14, width=2, length=8)

base_colors = {'A': 'red', 'C': 'orange', 'T': 'blue', 'G': 'green'}
legend_bases = [Patch(color=base_colors[base], label=base) for base in ['A', 'T', 'G', 'C']]
# legend_total = legend_bases + legend_areas
ax.legend(handles=legend_bases , title="Bases", loc='upper left')


arquivo_png = "/content/drive/MyDrive/pesquisa/desenho_primer_right_jusante.png"
plt.savefig(arquivo_png, bbox_inches='tight', pad_inches=0, dpi=300)

plt.show()

In [ ]:
# PASSO 11 - Conferencia dos amplicons - BLASTn - gera arquivos BLASTN
# BLASTn -> Basic Local Alignment Search Tool for nucleotides.
# BLASTN com sleep para evitar sobrecarga no servidor do blastn
#    => com opções de restart e/ou parada forçada
# 16-09-2025 17-09-2025 18-09-2025 index: indice do arquivo FIM lido
!pip install biopython

import pandas as pd
import time
from datetime import datetime
from Bio.Blast import NCBIWWW, NCBIXML
# Bio.Blast.email = "lucio.martins@gmail.com"

restart_pos = -1 # -1
parada_forcada = -1

total_alinhamentos = 0

arquivo_selecionados = pasta_analise + 'FIM_full.csv'
df_selecionados = pd.read_csv(arquivo_selecionados, delimiter = ";")
print(f"> lido: {arquivo_selecionados} ({len(df_selecionados)})")

for index, row in df_selecionados.iterrows():
  if parada_forcada != -1 and index > parada_forcada:
    print(f"  > parada forçada: {parada_forcada}")
    break
# ============================
  amplicon = row['amplicon']
# ============================
  origem = row['origem']
  tipo = row['tipo']
  sequencial_primer = row['sequencial']

  if restart_pos != -1:
    if index == restart_pos:
      print(f' >> FIM RESTART {restart_pos} index: {index}')
      restart_pos = -1
    else:
      print(f' >> index: {index} - não processa - RESTART em {restart_pos}')
      continue

  data1 = datetime.now()
  print(f"  > index processamento: {index} - {index + 1} de {len(df_selecionados)} origem: {origem} tipo: {tipo} sequencial_primer: {sequencial_primer} tamanho: {len(amplicon)} - início: {data1}")
  print(f"    > BlastN - processar amplicon (index: {index}): {amplicon}")

  try:
    # execução básica do blast
    # result_handle = NCBIWWW.qblast("blastn", "nt", amplicon)

    # execução avançada do blast
    result_handle = NCBIWWW.qblast(
        program="blastn",
        database="nt",
        sequence=amplicon,
        expect=10.0,
        hitlist_size=50,
        gapcosts="5 2",
        word_size=11,
        filter="L",
        alignments=50,
        format_type="XML",
        # aki testando as configuracoes
        # https://www.ncbi.nlm.nih.gov/datasets/taxonomy/5061/
        # entrez_query = "txid5061[Organism]"
        # entrez_query = "Aspergillus niger (taxid:5061)[Organism] Whole-genome shotgun contigs (wgs)[database]"
        # entrez_query = "wgs[Filter]"
        entrez_query = "wgs[filter]"
        # entrez_query = "txid5061[Organism] AND wgs[Filter]"
        # entrez_query="Aspergillus[Organism] AND wgs[Filter]"
        # entrez_query="Aspergillus niger [Organism] AND wgs [Filter]"
        # entrez_query = "Aspergillus niger [Organism] AND Ascomycota [Organism]"
    )

    # resultado do blast
    blast_records = NCBIXML.read(result_handle)

    data2 = datetime.now()
    print(f"    > BlastN Processado - {data2} ({data2 - data1})")

    sequencial_alinhamento = 0
    alinhamentos = []
    for alignment in blast_records.alignments:
      for hsp in alignment.hsps:
        linha_completa = row.to_dict()
        linha_completa['sequencial_alinhamento'] = sequencial_alinhamento
        linha_completa['accession_number'] = alignment.accession
        linha_completa['alignment_title'] = alignment.title
        linha_completa['alignment_hit_id'] = alignment.hit_id
        linha_completa['alignment_hit_def'] = alignment.hit_def
        linha_completa['alignment_length'] = alignment.length
        linha_completa['score'] = hsp.score
        linha_completa['bits'] = hsp.bits
        linha_completa['expect'] = hsp.expect
        linha_completa['query'] = hsp.query
        linha_completa['match'] = hsp.match
        linha_completa['sbjct'] = hsp.sbjct
        linha_completa['query_start'] = hsp.query_start
        linha_completa['query_end'] = hsp.query_end
        linha_completa['sbjct_start'] = hsp.sbjct_start
        linha_completa['sbjct_end'] = hsp.sbjct_end
        linha_completa['identities'] = hsp.identities
        linha_completa['positives'] = hsp.positives
        linha_completa['gaps'] = hsp.gaps
        linha_completa['align_length'] = hsp.align_length
        linha_completa['percent_identity'] = (hsp.identities / hsp.align_length) * 100

        sequencial_alinhamento = sequencial_alinhamento + 1
        alinhamentos.append(linha_completa)


    texto = f'      > total de alinhamentos: {len(alinhamentos)}'
    if len(alinhamentos) == 0:
      texto += " -> sem alinhamentos para este amplicon"
    print(texto)
    if len(alinhamentos) > 0:
      df_alinhamentos = pd.DataFrame(alinhamentos)
      url_arquivo = pasta_analise + f'{origem}_BLASTN_{tipo}-{index}.csv'
      df_alinhamentos.to_csv(url_arquivo, sep=';', index=False)
      print(f"        ⚡⚡⚡");
      print(f'      > gravado: {url_arquivo} ({len(df_alinhamentos)})')
    else:
      print(f'99999 > sem alinhamentos para este amplicon')

    time.sleep(10)  # espera 10 segundos entre requisições
  except Exception as e:
    print(f"   >>> Erro na sequência {index}: {e}")
    continue

print(f'>>> fim')

# Um HSP representa um alinhamento local entre duas sequências — a query (consulta) e a subject (alvo no banco de dados) — que obteve uma pontuação
# elevada com base nos critérios do BLAST. Ele é o núcleo do resultado do BLAST, mostrando onde há similaridade significativa entre as sequências.

# Características de um HSP:
#     Pode conter mismatches (bases diferentes) e gaps (lacunas), mas ainda assim ter uma pontuação alta.
#     É gerado a partir da extensão de uma correspondência inicial (seed) que o BLAST encontra.
#     Cada alinhamento entre query e subject pode ter vários HSPs, se houver múltiplas regiões semelhantes.

# Percent Identity:  Representa a porcentagem de bases idênticas entre a sequência de consulta (query) e a sequência alvo (subject) dentro da região alinhada.
# Outros campos importantes que complementam o alinhamento:
#     Alignment Length: número total de bases alinhadas (inclui matches, mismatches e gaps)
#     E-value: probabilidade de o alinhamento ocorrer por acaso (quanto menor, melhor)
#     Bit Score: qualidade do alinhamento (quanto maior, melhor)
# Atributo	Descrição
# score	Pontuação bruta do alinhamento
# bits	Bit score (pontuação normalizada)
# expect	E-value (probabilidade de alinhamento por acaso)
# query	Sequência da consulta no alinhamento
# match	String de correspondência (`	`, espaço, etc.) entre query e subject
# sbjct	Sequência da base de dados (subject) no alinhamento
# query_start	Posição inicial da sequência de consulta
# query_end	Posição final da sequência de consulta
# sbjct_start	Posição inicial da sequência alvo
# sbjct_end	Posição final da sequência alvo
# identities	Número de bases idênticas no alinhamento
# positives	Número de posições com substituições conservadas (em BLASTP)
# gaps	Número de lacunas no alinhamento
# align_length	Comprimento total do alinhamento

# Característica	megablast	e blastn
# = Foco principal: Sequências muito semelhantes (intraspécie) ->	Sequências divergentes (interespécie)
# = Velocidade:	Muito mais rápido ->Mais lento
# = Sensibilidade:	Menor (pode perder alinhamentos curtos ou com erros) ->Maior (detecta alinhamentos mais fracos)
# = Tamanho da palavra: Grande (ex: 28 bp) ->Menor (ex: 11 bp)
# = Ideal para: Genomas da mesma espécie, alta identidade -> Genes conservados entre espécies diferentes
# = Aplicações comuns:	WGS, genomas de referência -> Metagenômica, amplicons curtos, variantes

In [ ]:
# PASSO 11.1 - junta arquivos BLASTN

import pandas as pd

arquivos_primer = busca_arquivos('BLASTN')

dados = []
print(f'quantidade_arquivos: {len(arquivos_primer)}')
for idx, arquivo in enumerate(arquivos_primer):
  df_arq_primer = pd.read_csv(arquivo, delimiter = ";")
  print(f'  > lido: {arquivo} {len(df_arq_primer)}')
  dados.append(df_arq_primer)
  print(f'  > lido: {arquivo} {len(df_arq_primer)}')
df = pd.concat(dados, ignore_index=True)
url_arquivo = pasta_analise + 'RESULT.csv'
df.to_csv(url_arquivo, sep=';', index=False)
print(f'>>> FIM > gravado: {url_arquivo} ({len(df)})')
df

In [ ]:
# PASSO 11.2 - visualizar blastN - ler o arquivo RESULT

import pandas as pd

url_arquivo_result = pasta_analise + 'RESULT.csv'
df_arquivo_result = pd.read_csv(url_arquivo_result, delimiter = ";")

cols_result = [ 'alinhamento',
                'alignment_title',
                'alignment_hit_id',
                # 'alignment_hit_def',
                # 'alignment_length',
                # 'score',
                # 'bits',
                # 'expect',
                # 'query',
                # 'match',
                # 'sbjct',
                # 'query_start',
                # 'query_end',
                # 'sbjct_start',
                # 'sbjct_end',
                # 'identities',
                # 'positives',
                # 'gaps',
                # 'align_length',
                'percent_identity']

arquivo_result_fim = pd.DataFrame(df_arquivo_result, columns=cols_result)

print(f"{arquivo_result_fim}")

In [ ]:
# Passo 11.3 - filtro arquivo RESULT -> RESULT_FILTRO

import pandas as pd

url_arquivo = pasta_analise + 'RESULT.csv'
df = pd.read_csv(url_arquivo, delimiter = ";")
print(f'>>> lido: {url_arquivo} ({len(df)})')

df_q = df.query('percent_identity > 89')

arquivo_saida = pasta_analise + 'RESULT_FILTRO.csv'
df_q.to_csv(arquivo_saida, sep=';', index=False)
print(f'>>> FIM > gravado: {arquivo_saida} ({len(df_q)})')


In [ ]:
# Executar chamada ao primer-blast web

!pip install selenium
!pip install selenium webdriver-manager

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configurar o driver (certifique-se de ter o ChromeDriver instalado)
driver = webdriver.Chrome()

# Abrir o site do Primer-BLAST
driver.get("https://www.ncbi.nlm.nih.gov/tools/primer-blast/")

# Preencher os campos dos primers
primer1 = "ATCGTACGATCGTACG"
primer2 = "CGTACGTAGCTAGCTA"

driver.find_element(By.ID, "PRIMER1").send_keys(primer1)
driver.find_element(By.ID, "PRIMER2").send_keys(primer2)

# Submeter o formulário
driver.find_element(By.ID, "submit_button").click()

# Aguardar o carregamento dos resultados (até 2 minutos)
try:
    print("⏳ Aguardando resultados do Primer-BLAST...")
    WebDriverWait(driver, 120).until(
        EC.presence_of_element_located((By.ID, "content"))
    )
    print("✅ Resultados carregados com sucesso!")

    # Capturar o conteúdo da página de resultados
    html_resultado = driver.page_source

    # Salvar o HTML em um arquivo local
    with open("resultado_primerblast.html", "w", encoding="utf-8") as f:
        f.write(html_resultado)
    print("📄 Resultado salvo em 'resultado_primerblast.html'")

except Exception as e:
    print("❌ Erro ao carregar os resultados:", e)

# Fechar o navegador
driver.quit()

In [ ]:
# inclui previsao de temperatura de melting e amostra
# amostra sera usada em seguida para gerar o flankeamento

import primer3

arquivo = url_arquivo = pasta_analise + 'EXCLUSIVOS.csv'
ds = pd.read_csv(arquivo, delimiter = ";")
ds['amostra'] = ds['unidade'] * ds['copias']
ds['tm'] = ds.apply(lambda row: primer3.calc_tm(row['amostra']), axis=1)

url_arquivo = pasta_analise + 'EXCLUSIVOS_DET.csv'
ds.to_csv(url_arquivo, sep=';', index=False)
print(url_arquivo)

In [ ]:
# ler todos os arquivos

import pandas as pd

arquivos_primer = busca_arquivos('SELECT')

dados = []
print(f'quantidade_arquivos: {len(arquivos_primer)}')
for idx, arquivo in enumerate(arquivos_primer):
  df_arq_primer = pd.read_csv(arquivo, delimiter = ";")
  print(f'  > lido: {arquivo} {len(df_arq_primer)}')
#   dados.append(df_arq_primer)
#   print(f'  > lido: {arquivo} {len(df_arq_primer)}')
# df = pd.concat(dados, ignore_index=True)
# print(f'>>> fim')
# df

In [ ]:
# apaga arquivos - exclui arquivos - deleta arquivos
import os
import datetime

# arquivos_str = busca_arquivos('Alignment', extensao='xml', ordenar_por="nome", reverso=True)
arquivos_str = busca_arquivos('BLASTN', extensao='csv', ordenar_por="nome", reverso=True)

print(f'quantidade_arquivos: {len(arquivos_str)}')

for idx_arquivo, arquivo in enumerate(arquivos_str):
  if os.path.exists(arquivo):
    timestamp_criacao = os.path.getctime(arquivo)
    data_hora_criacao = datetime.datetime.fromtimestamp(timestamp_criacao)
    print(f'{arquivo} {data_hora_criacao}')
    # os.remove(arquivo)
    # df = pd.read_csv(arquivo, delimiter = ";")
    # print(f'{arquivo} ({len(df)})')

In [ ]:
import matplotlib.pyplot as plt

# Dados simulados de alinhamentos BLASTn
alinhamentos = [
    {
        "sample": "Ladder",
        "match": "|" * 50,  # bandas padrão
        "bands": [100, 200, 300, 400, 500]  # posições fixas
    },
    {
        "sample": "Sample 1",
        "match": "||||||||||||||||||||||||||||||||",
    },
    {
        "sample": "Sample 2",
        "match": "|||||| ||||||| |||||| |||||||||",
    },
    {
        "sample": "Sample 3",
        "match": "||||| ||||||| |||||| ||||||||||",
    },
    {
        "sample": "Sample 4",
        "match": "|||||||   ||||||||||            ",
    },
    {
        "sample": "Sample 5",
        "match": "||||||||||||||||||||            ",
    },
]

# Criar figura
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_facecolor('black')

# Desenhar bandas
for i, aln in enumerate(alinhamentos):
    x = i * 1.5
    sample = aln["sample"]

    if sample == "Ladder":
        for band in aln["bands"]:
            ax.add_patch(plt.Rectangle((x, band), 1.0, 10, color='white'))
            ax.text(x + 0.5, band + 15, f'{band} bp', ha='center', color='white', fontsize=8)
    else:
        match_count = aln["match"].count('|')
        align_len = len(aln["match"])
        brilho = match_count / align_len
        cor = (brilho, brilho, brilho)
        band_pos = match_count * 10  # escala fictícia

        ax.add_patch(plt.Rectangle((x, band_pos), 1.0, 10, color=cor))
        ax.text(x + 0.5, band_pos + 15, f'{match_count} bp', ha='center', color='white', fontsize=8)

    ax.text(x + 0.5, 20, sample, ha='center', color='white', fontsize=10)

# Estética
ax.set_xlim(-0.5, len(alinhamentos) * 1.5)
ax.set_ylim(0, 600)
ax.set_xticks([])
ax.set_yticks([100, 200, 300, 400, 500])
ax.set_yticklabels([f'{y} bp' for y in [100, 200, 300, 400, 500]], color='white')
ax.set_title("🧬 Simulação de Gel PCR com Referência e Alinhamentos", fontsize=14, color='white')
plt.tight_layout()
plt.show()

In [ ]:
#  Reuniao 20-10-2025 - BLASTN
!pip install biopython

path_file = pasta_analise + 'RESULT_FILTRO.csv'
df = pd.read_csv(path_file, delimiter = ";")

In [ ]:
df_filtro = df
df_filtro['align_length'] = pd.to_numeric(df_filtro['align_length'], errors='coerce')

# df_filtro = df.query('percent_identity == 100 & alignment_length > 199')
df_filtro = df.query('align_length > 199.99')
print(len(df_filtro))
# df_filtro['align_length']
# df_filtro.iloc[0]['amplicon']

In [ ]:
len(df)

In [ ]:
#  Reuniao 20-10-2025 - BLASTN
!pip install biopython
# Database: Whole-genome shotgun contigs (wgs)
# Organism: Aspergillus niger (taxid:5061)

import pandas as pd
import time
from datetime import datetime
from Bio.Blast import NCBIWWW, NCBIXML
# Bio.Blast.email = "lucio.martins@gmail.com"

# =====================#
row = df_filtro.iloc[23]
# =====================#

sequencia_teste = row['amplicon']
sequencia_teste = 'CGCTAGCGGTAGGAACAGCAGGGACAGAGCCGGACGGACCGGCGGGAACAACGCTACCAGAGGGAGCAGCACCTCCAGAGCCAGAGCCAGAGCCGGACCAGGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCAGAGCCGCCAGTAGAACCACCAGTAGAGCCGCCAGCACCACCACCGACGATGGGGGTAACGCTAGGAGCACCAGACTGAGCAGGGGCACTAGGAGCAGAACCGCCCTCGGTCATAGTAGCAACGGGAACAACAGTCTCGCCGGTGATAGTGTCGCAGGGAGTAGTGGAGGCACCGGCACTGGCGCCTTCCGGAACAGAGGCAG'
print(sequencia_teste)

data1 = datetime.now()

# execução básica do blast

# execução avançada do blast
# result_handle = NCBIWWW.qblast("blastn", "nt", sequencia_teste)
result_handle = NCBIWWW.qblast(
    program="blastn", # → usa megablast por padrão
    database="nt",
    hitlist_size=200,
    # expect=0.001,
    sequence=sequencia_teste,
    # aki testando as configuracoes
    # https://www.ncbi.nlm.nih.gov/datasets/taxonomy/5061/
    entrez_query = "txid5061[Organism]"
    # entrez_query = "txid5061[Organism]"
    # entrez_query = "wgs[Filter]"
    # entrez_query = "txid5061[Organism] AND wgs[Filter]"
    # entrez_query="Aspergillus[Organism] AND wgs[Filter]"
    # entrez_query="Aspergillus niger [Organism] AND wgs [Filter]"
    # entrez_query = "Aspergillus niger [Organism] AND Ascomycota [Organism]"
)

# resultado do blast
blast_records = NCBIXML.read(result_handle)

data2 = datetime.now()
print(f"> BlastN Processado - {data2} ({data2 - data1})")

sequencial_alinhamento = 0
alinhamentos = []
for alignment in blast_records.alignments:
  for hsp in alignment.hsps:
    linha_completa = row.to_dict()
    linha_completa['sequencial_alinhamento'] = sequencial_alinhamento
    linha_completa['accession_number'] = alignment.accession
    linha_completa['alignment_title'] = alignment.title
    linha_completa['alignment_hit_id'] = alignment.hit_id
    linha_completa['alignment_hit_def'] = alignment.hit_def
    linha_completa['alignment_length'] = alignment.length
    linha_completa['score'] = hsp.score
    linha_completa['bits'] = hsp.bits
    linha_completa['expect'] = hsp.expect
    linha_completa['query'] = hsp.query
    linha_completa['match'] = hsp.match
    linha_completa['sbjct'] = hsp.sbjct
    linha_completa['query_start'] = hsp.query_start
    linha_completa['query_end'] = hsp.query_end
    linha_completa['sbjct_start'] = hsp.sbjct_start
    linha_completa['sbjct_end'] = hsp.sbjct_end
    linha_completa['identities'] = hsp.identities
    linha_completa['positives'] = hsp.positives
    linha_completa['gaps'] = hsp.gaps
    linha_completa['align_length'] = hsp.align_length
    linha_completa['percent_identity'] = (hsp.identities / hsp.align_length) * 100

    sequencial_alinhamento = sequencial_alinhamento + 1
    alinhamentos.append(linha_completa)
texto = f'> total de alinhamentos: {len(alinhamentos)}'
if len(alinhamentos) == 0:
  texto += "* sem alinhamentos para este sequencia_teste"
print(texto)
if len(alinhamentos) > 0:
  df_alinhamentos = pd.DataFrame(alinhamentos)
  dicionario = df_amostras.set_index("id")["Cepa (strain)"].to_dict()
  df_alinhamentos['cepa'] = df_alinhamentos['origem'].map(dicionario)

  url_arquivo = f'{pasta_analise}AVALIACAO_1.csv'
  df_alinhamentos.to_csv(url_arquivo, sep=';', index=False)
  print(f'  > gravado: {url_arquivo} ({len(df_alinhamentos)})')
else:
  print(f'99999 > sem alinhamentos para este sequencia_teste')
print(f'>>> fim')

In [ ]:
url_arquivo = f'{pasta_analise}AVALIACAO_1.csv'
df_alinhamentos = pd.read_csv(url_arquivo, delimiter = ";")

# colunas = ['cepa', 'origem', 'unidade', 'copias', 'align_length', 'alignment_hit_id', 'alignment_title', 'parte_localizada', 'percent_identity']
colunas = ['cepa', 'origem', 'unidade', 'copias', 'align_length', 'alignment_hit_id', 'alignment_title']
# df_query = df_alinhamentos.query('percent_identity > 90')
# df_query[colunas]
df_alinhamentos[colunas]

In [ ]:
import re

def numero_inteiro_encontrado(row):
    cepa = str(row['cepa'])
    titulo = str(row['alignment_title']).lower()

    # Extrai todos os números da cepa (inteiros e decimais)
    numeros = re.findall(r'\d+(?:\.\d+)?', cepa)

    # Mantém apenas a parte inteira de cada número
    partes_inteiras = [num.split('.')[0] for num in numeros]

    # Verifica se alguma parte inteira está no título
    for parte in partes_inteiras:
        if parte in titulo:
            return parte  # Retorna a primeira parte encontrada

    return None  # Se nada for encontrado

# df_alinhamentos.at[0, 'alignment_title'] += df_alinhamentos.at[0, 'cepa'] # teste - força achado
# Aplicar ao DataFrame
df_alinhamentos['parte_localizada'] = df_alinhamentos.apply(numero_inteiro_encontrado, axis=1)

# Filtrar registros com correspondência
df_com_match_num = df_alinhamentos[df_alinhamentos['parte_localizada'].notna()]


df_com_match_num[colunas]

In [ ]:
def palavra_cepa_encontrada(row):
    cepa = str(row['cepa']).lower().strip()
    titulo = str(row['alignment_title']).lower().strip()

    # Divide a cepa em palavras e verifica se alguma está no título
    for palavra in cepa.split():
        if palavra in titulo:
            return palavra  # Retorna a primeira palavra encontrada
    return None  # Nenhuma correspondência

# Aplica a função ao DataFrame
df_alinhamentos_pos = pd.DataFrame(df_alinhamentos)
df_alinhamentos_pos['match_cepa'] = df_alinhamentos_pos.apply(palavra_cepa_encontrada, axis=1)

# Filtra os registros com correspondência
df_com_match = df_alinhamentos_pos[df_alinhamentos_pos['match_cepa'].notna()]

# Exibe apenas as colunas desejada
colunas = ['cepa', 'origem', 'unidade', 'copias', 'align_length', 'alignment_hit_id', 'alignment_title', 'match_cepa', 'percent_identity']
df_com_match[colunas]

In [ ]:
# reunir os arquivos de imagem em diretorio especifico

import os
import shutil

arquivos_str = busca_arquivos('schema', extensao='png', ordenar_por="nome", reverso=True)
print(f'quantidade_arquivos: {len(arquivos_str)}')

for idx_arquivo, arquivo in enumerate(arquivos_str):
    origem = os.path.basename(arquivo).split('_')[0]
    pasta_imagens = os.path.join(pasta_analise, 'imagens', origem)
    print(f'pasta_imagens: {pasta_imagens}')

    if not os.path.exists(pasta_imagens):
        os.makedirs(pasta_imagens)

    shutil.move(arquivo, pasta_imagens)
print('>>> fim')

In [ ]:
colunas = ['cepa', 'origem', 'unidade', 'copias', 'align_length', 'alignment_hit_id', 'alignment_title', 'parte_localizada', 'percent_identity']
df_query = df_alinhamentos.query('percent_identity == 100')
df_query[colunas]



In [ ]:
df_alinhamentos.at[0, 'alignment_title'] += df_alinhamentos.at[0, 'cepa']
df_alinhamentos.iloc[0]['alignment_title']